In [ ]:
# === Cell 1: 환경 설정 & Earth Engine 초기화 ===
import os, json, warnings
from pathlib import Path
from datetime import datetime, date, timedelta

import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

warnings.filterwarnings("ignore")

# ===== 디렉터리 설정 =====
NB_DIR   = Path.cwd()
OUT_DIR  = NB_DIR / "outputs"
ASSETS   = OUT_DIR / "web_assets"
OUT_DIR.mkdir(exist_ok=True)
ASSETS.mkdir(parents=True, exist_ok=True)

print("="*80)
print("NOTEBOOK 07: INTERACTIVE VISUALIZATIONS & DASHBOARD")
print("="*80)
print(f"\n📁 Directories:")
print(f"   Notebook dir: {NB_DIR}")
print(f"   Output dir:   {OUT_DIR}")
print(f"   Web assets:   {ASSETS}")

# ===== Earth Engine (선택 사항) =====
EE_PROJECT_ID = os.environ.get("EE_PROJECT_ID", "nasa-flood")
EE_READY = False

try:
    import ee
    try:
        ee.Initialize(project=EE_PROJECT_ID)
        EE_READY = True
        print(f"\n✅ Earth Engine initialized (project='{EE_PROJECT_ID}')")
    except Exception:
        print("\n🔐 Authenticating with Earth Engine...")
        ee.Authenticate()
        ee.Initialize(project=EE_PROJECT_ID)
        EE_READY = True
        print(f"✅ Authenticated & initialized (project='{EE_PROJECT_ID}')")
except Exception as e:
    print(f"\n⚠️  Earth Engine not available: {type(e).__name__}")
    print("   Will only load cached CSV/JSON (offline mode)")
    EE_READY = False

print(f"\n⏰ Generated: {datetime.now().isoformat(timespec='seconds')}")
print(f"🔧 EE Ready: {EE_READY}")
print("\n" + "="*80)

In [ ]:
# === Cell 2: 프로젝트 설정 & 유틸리티 함수 ===

# ===== AOI/윈도/임계값/이벤트 =====
if EE_READY:
    AOI_DELTA    = ee.Geometry.Rectangle([104.30,  8.50, 106.90, 10.90], geodesic=False)
    AOI_TONLESAP = ee.Geometry.Rectangle([103.30, 12.00, 105.20, 13.70], geodesic=False)

CFG = {
    "YEARS": list(range(2015, 2025)),
    "FLOOD_MONTHS": (8, 9),    # Aug–Sep
    "DROUGHT_MONTHS": (3, 4),  # Mar–Apr
    "TH_VV_DB": -16.0,
    "TH_VH_DB": -22.0,
    "TH_UNCERTAINTY_DB": 2.0,  # ±2 dB uncertainty
    "BASELINE_YEARS": [2005, 2006, 2007, 2008],
    "EVENTS": {
        "JINGHONG_FLOW_CUT": "2019-07-15",
        "XIAOWAN_ONLINE":    "2009-01-01",
        "NUOZHADU_ONLINE":   "2012-01-01",
    },
}

FLOOD_M1, FLOOD_M2 = CFG["FLOOD_MONTHS"]
DRY_M1,   DRY_M2   = CFG["DROUGHT_MONTHS"]
EVENTS = {k: pd.to_datetime(v) for k, v in CFG["EVENTS"].items()}

# ===== 색상 팔레트 (일관성) =====
COLORS = {
    "vv": "#1f77b4",        # Blue
    "vh": "#ff7f0e",        # Orange
    "vh_only": "#d62728",   # Red
    "precip": "#2ca02c",    # Green
    "baseline": "#8c564b",  # Brown
    "event": "#7f7f7f",     # Gray
    "gain": "#9467bd",      # Purple
    "critical": "#d62728",  # Red
    "moderate": "#ff7f0e",  # Orange
    "fair": "#ffbb78",      # Light orange
    "healthy": "#2ca02c",   # Green
}

# ===== 공통 유틸 =====
def _daterange_of_year_months(year:int, m1:int, m2:int):
    """Return ISO date range for [m1..m2] months."""
    start = date(year, m1, 1)
    end   = (date(year+1,1,1) - timedelta(days=1)) if m2==12 else (date(year,m2+1,1)-timedelta(days=1))
    return start.isoformat(), end.isoformat()

# ===== Earth Engine 함수 (EE_READY일 때만) =====
if EE_READY:
    def s1_min(aoi, start, end, pol):
        """Sentinel-1 min composite."""
        return (ee.ImageCollection('COPERNICUS/S1_GRD')
                .filterBounds(aoi)
                .filterDate(start, end)
                .filter(ee.Filter.eq('instrumentMode','IW'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
                .select(pol).min().clip(aoi))

    def classify_water(img_min, pol, threshold_db):
        """Binary water classification."""
        return img_min.lt(threshold_db).selfMask()

    def area_km2(mask_img, aoi, scale=30, band_name='constant'):
        """Compute area in km²."""
        area = (mask_img.multiply(ee.Image.pixelArea())
                .reduceRegion(ee.Reducer.sum(), aoi, scale, maxPixels=1e12))
        return ee.Number(area.get(band_name)).divide(1e6)

    def chirps_sum_mm(aoi, start, end):
        """CHIRPS precipitation sum."""
        col = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
               .filterBounds(aoi).filterDate(start, end).select('precipitation'))
        if col.size().getInfo() == 0:
            return None
        total = col.sum().reduceRegion(ee.Reducer.mean(), aoi, 5000, maxPixels=1e12)
        return ee.Number(total.get('precipitation'))

print("✅ Config ready | Years:", CFG["YEARS"][0], "-", CFG["YEARS"][-1])
print("✅ Color palette defined for consistency")
print(f"✅ Utility functions loaded (EE_READY={EE_READY})")

In [ ]:
# === Cell 3: 데이터 로딩 (Smart Cache System with Validation) ===
"""
🎯 OBJECTIVE: Load data with fallback hierarchy

PRIORITY:
1. Cached files (07_web_assets)
2. Recent notebook outputs (01-06)
3. Earth Engine computation (if available)

IMPROVEMENTS:
- Multi-directory search
- Data validation
- Quality flags
- Fallback handling
"""

# ===== 경로 후보 생성기 =====
PROJECT_ROOT = NB_DIR.parent if (NB_DIR / "..").exists() else NB_DIR
CANDIDATE_DIRS = [
    ASSETS,                                           # 07 캐시
    OUT_DIR,                                          # notebooks/outputs
    PROJECT_ROOT / "outputs",                         # 프로젝트 루트
    PROJECT_ROOT / "streamlit_app" / "data" / "processed",  # 앱 데이터
]

print("\n" + "="*80)
print("SMART CACHE SYSTEM")
print("="*80)
print("\nSearch directories (in priority order):")
for i, d in enumerate(CANDIDATE_DIRS, 1):
    exists = "✓" if d.exists() else "✗"
    print(f"   {i}. {exists} {d}")

def read_csv_candidates(*names):
    """
    Load CSV from first available location.
    
    Args:
        *names: Candidate filenames to try
    
    Returns:
        pd.DataFrame or None
    """
    paths = [d / n for d in CANDIDATE_DIRS for n in names]
    for p in paths:
        if p.exists():
            try:
                df = pd.read_csv(p)
                print(f"   ✓ Loaded: {p.relative_to(PROJECT_ROOT)}")
                return df
            except Exception as e:
                print(f"   ⚠️ Failed to read {p.name}: {type(e).__name__}")
                continue
    
    print(f"   ❌ Not found: {list(names)}")
    return None

def read_json_candidates(*names):
    """Load JSON from first available location."""
    paths = [d / n for d in CANDIDATE_DIRS for n in names]
    for p in paths:
        if p.exists():
            try:
                with open(p, "r", encoding="utf-8") as f:
                    data = json.load(f)
                print(f"   ✓ Loaded: {p.relative_to(PROJECT_ROOT)}")
                return data
            except Exception as e:
                print(f"   ⚠️ Failed to read {p.name}: {type(e).__name__}")
                continue
    
    print(f"   ❌ Not found: {list(names)}")
    return None

# ===== EE 기반 빌더 (필요 시만 호출) =====
def _build_annual_flood_df(aoi, years):
    """Build flood dataset from Earth Engine (fallback)."""
    if not EE_READY:
        return None
    
    rows = []
    for y in years:
        start, end = _daterange_of_year_months(y, FLOOD_M1, FLOOD_M2)
        try:
            vv = s1_min(aoi, start, end, "VV")
            vh = s1_min(aoi, start, end, "VH")
            vv_mask = classify_water(vv, "VV", CFG["TH_VV_DB"])
            vh_mask = classify_water(vh, "VH", CFG["TH_VH_DB"])
            
            a_vv = float(area_km2(vv_mask, aoi, 30).getInfo() or 0.0)
            a_vh = float(area_km2(vh_mask, aoi, 30).getInfo() or 0.0)
            
            pr_n = chirps_sum_mm(aoi, start, end)
            pr = float(pr_n.getInfo() or 0.0) if pr_n else 0.0
            
            rows.append({
                "year": y, 
                "flood_vv_km2": a_vv, 
                "flood_vh_km2": a_vh, 
                "precip_wet_mm": pr
            })
        except Exception as e:
            print(f"      ⚠️ {y} failed: {type(e).__name__}")
            rows.append({
                "year": y,
                "flood_vv_km2": np.nan,
                "flood_vh_km2": np.nan,
                "precip_wet_mm": np.nan
            })
    
    return pd.DataFrame(rows)

def _build_annual_dry_df(aoi, years):
    """Build dry season dataset from Earth Engine (fallback)."""
    if not EE_READY:
        return None
    
    rows = []
    for y in years:
        start, end = _daterange_of_year_months(y, DRY_M1, DRY_M2)
        try:
            vh = s1_min(aoi, start, end, "VH")
            vh_mask = classify_water(vh, "VH", CFG["TH_VH_DB"])
            a_vh = float(area_km2(vh_mask, aoi, 30).getInfo() or 0.0)
            
            pr_n = chirps_sum_mm(aoi, start, end)
            pr = float(pr_n.getInfo() or 0.0) if pr_n else 0.0
            
            rows.append({"year": y, "dry_vh_km2": a_vh, "precip_dry_mm": pr})
        except Exception as e:
            print(f"      ⚠️ {y} failed: {type(e).__name__}")
            rows.append({"year": y, "dry_vh_km2": np.nan, "precip_dry_mm": np.nan})
    
    return pd.DataFrame(rows)

# ===== 스마트 로더 (검증 포함) =====
def smart_load_flood(aoi_name: str):
    """
    Load flood dataset with validation.
    
    Priority:
    1. Cached CSV
    2. Notebook 04/05 outputs
    3. Notebook 02 outputs
    4. Earth Engine computation
    
    Returns:
        pd.DataFrame with standardized columns
    """
    lower = aoi_name.lower().replace(" ", "_")
    
    # 1) Try cached/existing files
    df = read_csv_candidates(
        f"annual_flood_{lower}.csv",                     # 07 cache
        f"dualpol_comprehensive_2015_2024.csv",          # 05 output
        f"flood_extent_{lower}_vv_vh_2015_2024.csv",     # 04 output
        f"annual_analysis_{lower}.csv",                  # 02 output
    )
    
    if df is not None:
        # Standardize column names
        rename = {
            "vv_km2": "flood_vv_km2",
            "vh_km2": "flood_vh_km2",
            "precip_mm": "precip_wet_mm",
            "precipitation_mm": "precip_wet_mm",
        }
        df = df.rename(columns=rename)
        
        # Filter by AOI if multi-region file
        if "aoi" in df.columns:
            df = df[df["aoi"].str.lower().str.replace(" ", "_") == lower].copy()
        
        # Ensure required columns
        required = ["year", "flood_vv_km2", "flood_vh_km2", "precip_wet_mm"]
        for col in required:
            if col not in df.columns:
                df[col] = np.nan
        
        df = df[required].drop_duplicates(subset="year").sort_values("year")
        
        # Cache for future use
        cache_path = ASSETS / f"annual_flood_{lower}.csv"
        df.to_csv(cache_path, index=False)
        print(f"   💾 Cached → {cache_path.name}")
        
        return df
    
    # 2) Try Earth Engine computation
    if EE_READY:
        print(f"   🔨 Building from Earth Engine: {aoi_name}")
        aoi = AOI_DELTA if aoi_name == "Mekong_Delta" else AOI_TONLESAP
        df = _build_annual_flood_df(aoi, CFG["YEARS"])
        
        if df is not None:
            cache_path = ASSETS / f"annual_flood_{lower}.csv"
            df.to_csv(cache_path, index=False)
            print(f"   💾 Saved → {cache_path.name}")
            return df
    
    # 3) No data available
    print(f"   ❌ CRITICAL: No flood dataset for {aoi_name}")
    return pd.DataFrame(columns=["year", "flood_vv_km2", "flood_vh_km2", "precip_wet_mm"])

def smart_load_dry(aoi_name: str):
    """Load dry season dataset with validation."""
    lower = aoi_name.lower().replace(" ", "_")
    
    df = read_csv_candidates(
        f"annual_dry_{lower}.csv",                  # 07 cache
        "dry_season_analysis_2015_2024.csv",        # 06 output
    )
    
    if df is not None:
        if "aoi" in df.columns:
            df = df[df["aoi"].str.lower().str.replace(" ", "_") == lower].copy()
        
        rename = {
            "water_extent_km2": "dry_vh_km2",
            "precip_total_mm": "precip_dry_mm",
        }
        df = df.rename(columns=rename)
        
        required = ["year", "dry_vh_km2", "precip_dry_mm"]
        for col in required:
            if col not in df.columns:
                df[col] = np.nan
        
        df = df[required].drop_duplicates(subset="year").sort_values("year")
        
        cache_path = ASSETS / f"annual_dry_{lower}.csv"
        df.to_csv(cache_path, index=False)
        print(f"   💾 Cached → {cache_path.name}")
        
        return df
    
    if EE_READY:
        print(f"   🔨 Building from Earth Engine: {aoi_name}")
        aoi = AOI_DELTA if aoi_name == "Mekong_Delta" else AOI_TONLESAP
        df = _build_annual_dry_df(aoi, CFG["YEARS"])
        
        if df is not None:
            cache_path = ASSETS / f"annual_dry_{lower}.csv"
            df.to_csv(cache_path, index=False)
            print(f"   💾 Saved → {cache_path.name}")
            return df
    
    print(f"   ❌ CRITICAL: No dry dataset for {aoi_name}")
    return pd.DataFrame(columns=["year", "dry_vh_km2", "precip_dry_mm"])

# ===== 실제 로드 =====
print("\n" + "-"*80)
print("LOADING DATA")
print("-"*80)

print("\n📊 Flood datasets:")
df_delta_flood = smart_load_flood("Mekong_Delta")
df_ts_flood    = smart_load_flood("Tonle_Sap")

print("\n📊 Dry season datasets:")
df_delta_dry   = smart_load_dry("Mekong_Delta")
df_ts_dry      = smart_load_dry("Tonle_Sap")

# ===== 머지 및 파생 지표 =====
print("\n" + "-"*80)
print("MERGING & COMPUTING DERIVED METRICS")
print("-"*80)

df_delta = pd.merge(df_delta_flood, df_delta_dry, on="year", how="outer").sort_values("year")
df_ts    = pd.merge(df_ts_flood,    df_ts_dry,    on="year", how="outer").sort_values("year")

for df, name in [(df_delta, "Delta"), (df_ts, "Tonlé")]:
    # VH gain
    if {"flood_vh_km2", "flood_vv_km2"}.issubset(df.columns):
        df["vh_gain_km2"] = df["flood_vh_km2"] - df["flood_vv_km2"]
        df["vh_gain_pct"] = (df["vh_gain_km2"] / df["flood_vv_km2"] * 100).replace([np.inf, -np.inf], np.nan)
    
    print(f"\n{name}: {len(df)} years, {df.isna().sum().sum()} total NaNs")

print("\nSample data (Delta):")
display(df_delta.head(3))

print("\n" + "="*80)

In [ ]:
# === Cell 4: Baseline 로딩 (Fallback & Validation) ===
"""
🎯 OBJECTIVE: Load pre-dam baselines with robust fallback

SOURCES:
1. baseline_summary.json (Notebook 01)
2. Hardcoded fallback values (literature)

VALIDATION:
- Check data completeness
- Verify reasonable ranges
- Flag missing values
"""

print("\n" + "="*80)
print("BASELINE LOADING & VALIDATION")
print("="*80)

def load_or_fallback_baselines():
    """
    Load baselines with validation and fallback.
    
    Returns:
        dict with baseline values for each AOI
    """
    # Try to load from Notebook 01 output
    baseline_json = read_json_candidates("baseline_summary.json")
    
    if baseline_json and "areas" in baseline_json:
        base = {}
        for a in baseline_json["areas"]:
            aoi_name = a["aoi"]
            base[aoi_name] = {
                "wet_km2": float(a.get("baseline_wet_km2", 0.0)),
                "dry_km2": float(a.get("baseline_dry_km2", 0.0)),
                "source": "Landsat5_2005_2008",
                "validated": True
            }
        
        # Validate ranges
        for aoi_name, values in base.items():
            if values["wet_km2"] <= 0 or values["dry_km2"] <= 0:
                print(f"   ⚠️  {aoi_name}: Invalid baseline values (≤0)")
                values["validated"] = False
            elif values["wet_km2"] < values["dry_km2"]:
                print(f"   ⚠️  {aoi_name}: Wet < Dry (physically impossible)")
                values["validated"] = False
            else:
                print(f"   ✓ {aoi_name}: Wet={values['wet_km2']:,.0f}, Dry={values['dry_km2']:,.0f} km²")
        
        return base
    
    # Fallback to literature values
    print("   ⚠️  baseline_summary.json not found → using literature estimates")
    return {
        "Mekong_Delta": {
            "wet_km2": 8500.0,   # MRC estimates
            "dry_km2": 3200.0,
            "source": "MRC_literature",
            "validated": False
        },
        "Tonle_Sap": {
            "wet_km2": 12000.0,  # Kummu et al. (2014)
            "dry_km2": 2800.0,
            "source": "Kummu_2014",
            "validated": False
        },
    }

BASELINES = load_or_fallback_baselines()

# Add ecological thresholds for Tonle Sap
BASELINES["Tonle_Sap"]["ecological_thresholds"] = {
    "critical": 2000,
    "moderate": 2500,
    "optimal": 3000,
}

print("\n📊 Baseline Summary:")
for aoi, vals in BASELINES.items():
    status = "✓ Validated" if vals["validated"] else "⚠️ Literature"
    print(f"\n{aoi} ({vals['source']}) {status}:")
    print(f"   Wet season:  {vals['wet_km2']:>8,.0f} km²")
    print(f"   Dry season:  {vals['dry_km2']:>8,.0f} km²")
    
    if "ecological_thresholds" in vals:
        print(f"   Ecological thresholds:")
        for level, threshold in vals["ecological_thresholds"].items():
            print(f"      {level.capitalize():10}: {threshold:>6,} km²")

print("\n" + "="*80)

In [ ]:
# === Cell 5: 데이터 검증 & 품질 체크 ===
"""
🎯 OBJECTIVE: Validate loaded data before visualization

CHECKS:
1. Missing years
2. NaN values
3. Value ranges (physical plausibility)
4. VH ≥ VV consistency
5. Data completeness score
"""

print("\n" + "="*80)
print("DATA VALIDATION & QUALITY ASSESSMENT")
print("="*80)

def validate_dataset(df: pd.DataFrame, name: str, dataset_type: str):
    """
    Comprehensive data validation.
    
    Args:
        df: DataFrame to validate
        name: Dataset name (e.g., "Mekong Delta")
        dataset_type: "flood" or "dry"
    
    Returns:
        dict with validation results
    """
    issues = []
    warnings = []
    
    # 1) Missing years
    expected_years = set(CFG["YEARS"])
    actual_years = set(df["year"].dropna().astype(int).tolist())
    missing_years = sorted(expected_years - actual_years)
    
    if missing_years:
        issues.append(f"Missing years: {missing_years}")
    
    # 2) NaN analysis
    nan_summary = {}
    for col in df.columns:
        nan_count = df[col].isna().sum()
        if nan_count > 0:
            nan_summary[col] = f"{nan_count}/{len(df)}"
    
    if nan_summary:
        warnings.append(f"NaN values: {nan_summary}")
    
    # 3) Value range checks
    if dataset_type == "flood":
        # VV should be reasonable
        if "flood_vv_km2" in df.columns:
            vv_max = df["flood_vv_km2"].max()
            if vv_max > 50000:
                warnings.append(f"VV very large: {vv_max:,.0f} km² (check units)")
            if (df["flood_vv_km2"] < 0).any():
                issues.append("VV has negative values")
        
        # VH should be reasonable
        if "flood_vh_km2" in df.columns:
            vh_max = df["flood_vh_km2"].max()
            if vh_max > 60000:
                warnings.append(f"VH very large: {vh_max:,.0f} km² (check units)")
            if (df["flood_vh_km2"] < 0).any():
                issues.append("VH has negative values")
        
        # VH ≥ VV physical consistency
        if {"flood_vv_km2", "flood_vh_km2"}.issubset(df.columns):
            violations = df[(df["flood_vh_km2"].notna()) & 
                           (df["flood_vv_km2"].notna()) & 
                           (df["flood_vh_km2"] < df["flood_vv_km2"])]
            if len(violations) > 0:
                years = violations["year"].astype(int).tolist()
                warnings.append(f"VH<VV in years {years} (wind/roughness?)")
        
        # Precipitation plausibility
        if "precip_wet_mm" in df.columns:
            precip_max = df["precip_wet_mm"].max()
            if precip_max > 2000:
                warnings.append(f"Wet precip extreme: {precip_max:,.0f} mm (2 months)")
    
    elif dataset_type == "dry":
        # Dry season water extent
        if "dry_vh_km2" in df.columns:
            dry_max = df["dry_vh_km2"].max()
            if dry_max > 30000:
                warnings.append(f"Dry water very large: {dry_max:,.0f} km²")
            if (df["dry_vh_km2"] < 0).any():
                issues.append("Dry VH has negative values")
        
        # Dry precipitation
        if "precip_dry_mm" in df.columns:
            dry_precip_max = df["precip_dry_mm"].max()
            if dry_precip_max > 500:
                warnings.append(f"Dry precip high: {dry_precip_max:,.0f} mm (unusual)")
    
    # 4) Completeness score
    total_cells = len(df) * len(df.columns)
    valid_cells = total_cells - df.isna().sum().sum()
    completeness_pct = valid_cells / total_cells * 100 if total_cells > 0 else 0
    
    return {
        "name": name,
        "type": dataset_type,
        "issues": issues,
        "warnings": warnings,
        "completeness_pct": completeness_pct,
        "total_years": len(df),
        "valid_years": len(df.dropna(how="all"))
    }

# Validate all datasets
print("\n" + "-"*80)
print("FLOOD DATASETS")
print("-"*80)

val_delta_flood = validate_dataset(df_delta_flood, "Mekong Delta", "flood")
val_ts_flood = validate_dataset(df_ts_flood, "Tonlé Sap", "flood")

for val in [val_delta_flood, val_ts_flood]:
    print(f"\n{val['name']}:")
    print(f"   Completeness: {val['completeness_pct']:.1f}%")
    print(f"   Valid years: {val['valid_years']}/{val['total_years']}")
    
    if val['issues']:
        print(f"   ❌ ISSUES:")
        for issue in val['issues']:
            print(f"      • {issue}")
    
    if val['warnings']:
        print(f"   ⚠️  Warnings:")
        for warning in val['warnings']:
            print(f"      • {warning}")
    
    if not val['issues'] and not val['warnings']:
        print(f"   ✅ All checks passed")

print("\n" + "-"*80)
print("DRY SEASON DATASETS")
print("-"*80)

val_delta_dry = validate_dataset(df_delta_dry, "Mekong Delta", "dry")
val_ts_dry = validate_dataset(df_ts_dry, "Tonlé Sap", "dry")

for val in [val_delta_dry, val_ts_dry]:
    print(f"\n{val['name']}:")
    print(f"   Completeness: {val['completeness_pct']:.1f}%")
    print(f"   Valid years: {val['valid_years']}/{val['total_years']}")
    
    if val['issues']:
        print(f"   ❌ ISSUES:")
        for issue in val['issues']:
            print(f"      • {issue}")
    
    if val['warnings']:
        print(f"   ⚠️  Warnings:")
        for warning in val['warnings']:
            print(f"      • {warning}")
    
    if not val['issues'] and not val['warnings']:
        print(f"   ✅ All checks passed")

# Overall quality score
all_validations = [val_delta_flood, val_ts_flood, val_delta_dry, val_ts_dry]
avg_completeness = np.mean([v['completeness_pct'] for v in all_validations])
total_issues = sum(len(v['issues']) for v in all_validations)

print("\n" + "="*80)
print("OVERALL DATA QUALITY")
print("="*80)
print(f"\nAverage completeness: {avg_completeness:.1f}%")
print(f"Total critical issues: {total_issues}")

if avg_completeness >= 90 and total_issues == 0:
    print("\n✅ EXCELLENT: Data ready for high-quality visualization")
elif avg_completeness >= 70:
    print("\n⚠️  GOOD: Some missing data, proceed with caution")
else:
    print("\n❌ POOR: Significant data gaps, review before proceeding")

print("="*80)

In [ ]:
# === Cell 6: Plotting 유틸리티 (재사용 가능, 일관성) ===
"""
🎯 OBJECTIVE: Reusable plotting functions with consistent styling

IMPROVEMENTS:
- Consistent color scheme
- Responsive layouts
- Accessibility (contrast, labels)
- Error handling
- Metadata in plots
"""

def create_dual_subplot(title_left, title_right, height=500, width=1200):
    """
    Create dual subplot with consistent styling.
    
    Args:
        title_left: Left panel title
        title_right: Right panel title
        height: Figure height in pixels
        width: Figure width in pixels
    
    Returns:
        plotly Figure object
    """
    fig = make_subplots(
        rows=1, cols=2, 
        subplot_titles=(title_left, title_right), 
        horizontal_spacing=0.12
    )
    
    fig.update_layout(
        height=height, 
        width=width, 
        hovermode="x unified",
        font=dict(family="Arial, sans-serif", size=12),
        plot_bgcolor="white", 
        paper_bgcolor="white", 
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="gray",
            borderwidth=1
        )
    )
    
    # Grid styling
    fig.update_xaxes(
        dtick=1, 
        gridcolor="lightgray", 
        showline=True, 
        linewidth=1, 
        linecolor="black", 
        mirror=True
    )
    fig.update_yaxes(
        gridcolor="lightgray", 
        showline=True, 
        linewidth=1, 
        linecolor="black", 
        mirror=True
    )
    
    return fig

def add_event_markers(fig, events_dict, row=1, col=1):
    """
    Add vertical lines for key events.
    
    Args:
        fig: Plotly figure
        events_dict: Dict of {label: datetime}
        row: Subplot row
        col: Subplot column
    """
    for label, ts in events_dict.items():
        year = ts.year
        
        # Vertical line
        fig.add_vline(
            x=year, 
            line_dash="dot", 
            line_color=COLORS["event"], 
            opacity=0.6,
            row=row, col=col
        )
        
        # Annotation (only for main events)
        if "JINGHONG" in label:
            fig.add_annotation(
                x=year,
                y=0.95,
                text=f"⚠️ {year}<br>Jinghong",
                showarrow=False,
                yref="paper",
                xanchor="center",
                font=dict(size=9, color=COLORS["event"]),
                bgcolor="rgba(255,255,255,0.8)",
                bordercolor=COLORS["event"],
                borderwidth=1,
                row=row, col=col
            )

def add_baseline(fig, value, text, row=1, col=1, color=None):
    """
    Add horizontal baseline reference.
    
    Args:
        fig: Plotly figure
        value: Y-value for line
        text: Annotation text
        row: Subplot row
        col: Subplot column
        color: Line color (default: baseline color)
    """
    color = color or COLORS["baseline"]
    
    fig.add_hline(
        y=value, 
        line_dash="dash", 
        line_color=color, 
        opacity=0.7,
        row=row, col=col
    )
    
    fig.add_annotation(
        x=0.02,
        y=value,
        text=f"{text}: {value:,.0f} km²",
        showarrow=False,
        xref="paper",
        xanchor="left",
        font=dict(size=9, color=color),
        bgcolor="rgba(255,255,255,0.7)",
        row=row, col=col
    )

def finalize_figure(fig, title, save_json: Path=None, save_html: Path=None):
    """
    Finalize figure with title and save options.
    
    Args:
        fig: Plotly figure
        title: Main title
        save_json: Path to save JSON (for embedding)
        save_html: Path to save standalone HTML
    
    Returns:
        fig
    """
    fig.update_layout(
        title=dict(
            text=title, 
            x=0.5, 
            xanchor="center", 
            font=dict(size=16, weight="bold")
        )
    )
    
    # Save JSON for dashboard embedding
    if save_json is not None:
        save_json.parent.mkdir(parents=True, exist_ok=True)
        try:
            save_json.write_text(fig.to_json(), encoding="utf-8")
            print(f"   💾 JSON → {save_json.name}")
        except Exception as e:
            print(f"   ⚠️ JSON save failed: {type(e).__name__}")
    
    # Save standalone HTML
    if save_html is not None:
        save_html.parent.mkdir(parents=True, exist_ok=True)
        try:
            fig.write_html(str(save_html))
            print(f"   💾 HTML → {save_html.name}")
        except Exception as e:
            print(f"   ⚠️ HTML save failed: {type(e).__name__}")
    
    return fig

print("✅ Plotting utilities loaded")
print("   • Consistent color scheme")
print("   • Responsive layouts")
print("   • Event markers & baselines")
print("   • Dual save (JSON + HTML)")

In [ ]:
# === Cell 7: Figure 1 - Annual Flood Extent (VV vs VH) ===
"""
🎯 OBJECTIVE: Show VV/VH comparison over time

FEATURES:
- Dual panels (Delta + Tonlé Sap)
- VV and VH lines
- Baseline reference
- Event markers
- Uncertainty bands (optional)
"""

print("\n" + "="*80)
print("FIGURE 1: ANNUAL FLOOD EXTENT (VV vs VH)")
print("="*80)

def fig_annual_flood(df_delta, df_ts, baselines, events):
    """Create annual flood extent comparison."""
    
    fig = create_dual_subplot(
        "Mekong Delta — Flood Season (Aug–Sep)",
        "Tonlé Sap — Flood Season (Aug–Sep)",
        height=520, 
        width=1200
    )
    
    # ===== DELTA PANEL =====
    if {"year", "flood_vv_km2", "flood_vh_km2"}.issubset(df_delta.columns) and len(df_delta) > 0:
        valid_delta = df_delta.dropna(subset=["flood_vv_km2", "flood_vh_km2"])
        
        if len(valid_delta) > 0:
            # VV trace
            fig.add_trace(
                go.Scatter(
                    x=valid_delta["year"], 
                    y=valid_delta["flood_vv_km2"],
                    mode="lines+markers", 
                    name="Delta VV",
                    line=dict(color=COLORS["vv"], width=2.5),
                    marker=dict(size=6),
                    hovertemplate="Year: %{x}<br>VV: %{y:,.0f} km²<extra></extra>"
                ),
                row=1, col=1
            )
            
            # VH trace
            fig.add_trace(
                go.Scatter(
                    x=valid_delta["year"], 
                    y=valid_delta["flood_vh_km2"],
                    mode="lines+markers", 
                    name="Delta VH",
                    line=dict(color=COLORS["vh"], width=2.5),
                    marker=dict(size=6),
                    hovertemplate="Year: %{x}<br>VH: %{y:,.0f} km²<extra></extra>"
                ),
                row=1, col=1
            )
            
            # Baseline
            add_baseline(
                fig, 
                baselines["Mekong_Delta"]["wet_km2"], 
                "Pre-dam wet", 
                row=1, col=1
            )
            
            # Events
            add_event_markers(fig, events, row=1, col=1)
        else:
            fig.add_annotation(
                text="No valid data",
                xref="paper", yref="paper",
                x=0.25, y=0.5,
                showarrow=False,
                font=dict(size=14, color="gray"),
                row=1, col=1
            )
    
    # ===== TONLÉ SAP PANEL =====
    if {"year", "flood_vv_km2", "flood_vh_km2"}.issubset(df_ts.columns) and len(df_ts) > 0:
        valid_ts = df_ts.dropna(subset=["flood_vv_km2", "flood_vh_km2"])
        
        if len(valid_ts) > 0:
            fig.add_trace(
                go.Scatter(
                    x=valid_ts["year"], 
                    y=valid_ts["flood_vv_km2"],
                    mode="lines+markers", 
                    name="Tonlé VV",
                    line=dict(color=COLORS["vv"], width=2.5),
                    marker=dict(size=6),
                    hovertemplate="Year: %{x}<br>VV: %{y:,.0f} km²<extra></extra>"
                ),
                row=1, col=2
            )
            
            fig.add_trace(
                go.Scatter(
                    x=valid_ts["year"], 
                    y=valid_ts["flood_vh_km2"],
                    mode="lines+markers", 
                    name="Tonlé VH",
                    line=dict(color=COLORS["vh"], width=2.5),
                    marker=dict(size=6),
                    hovertemplate="Year: %{x}<br>VH: %{y:,.0f} km²<extra></extra>"
                ),
                row=1, col=2
            )
            
            add_baseline(
                fig, 
                baselines["Tonle_Sap"]["wet_km2"], 
                "Pre-dam wet", 
                row=1, col=2
            )
            
            add_event_markers(fig, events, row=1, col=2)
    
    # Axis labels
    fig.update_yaxes(title_text="Flood Area (km²)", row=1, col=1)
    fig.update_yaxes(title_text="Flood Area (km²)", row=1, col=2)
    fig.update_xaxes(title_text="Year", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=1, col=2)
    
    return finalize_figure(
        fig, 
        "Annual Flood Extent (VV vs VH, 2015–2024)",
        save_json=ASSETS / "fig_annual_flood.json",
        save_html=ASSETS / "fig_annual_flood.html"
    )

fig1 = fig_annual_flood(df_delta, df_ts, BASELINES, EVENTS)
fig1.show()

# Key metrics summary
print("\n📊 Key Metrics:")
for name, df in [("Delta", df_delta), ("Tonlé", df_ts)]:
    if "vh_gain_pct" in df.columns:
        avg_gain = df["vh_gain_pct"].mean()
        if not np.isnan(avg_gain):
            print(f"   {name:6} VH extra detection: {avg_gain:>5.1f}% (avg)")

print("\n" + "="*80)

In [ ]:
# === Cell 8: Figure 2 - VH Gain Analysis (Stacked Bar Chart) ===
"""
🎯 OBJECTIVE: Visualize "hidden flooding" detected by VH

DESIGN:
- Stacked bars: VV (base) + VH-only (gain)
- Shows absolute contribution
- Color-coded for clarity
- Gain percentage annotations
"""

print("\n" + "="*80)
print("FIGURE 2: VH GAIN ANALYSIS (STACKED BAR)")
print("="*80)

def fig_vh_gain_stacked(df_delta, df_ts):
    """Create stacked bar chart showing VH advantage."""
    
    fig = create_dual_subplot(
        "Mekong Delta — VV vs VH-only Contribution",
        "Tonlé Sap — VV vs VH-only Contribution",
        height=520,
        width=1200
    )
    
    # ===== DELTA PANEL =====
    if {"year", "flood_vv_km2", "vh_gain_km2"}.issubset(df_delta.columns) and len(df_delta) > 0:
        valid_delta = df_delta.dropna(subset=["flood_vv_km2", "vh_gain_km2"])
        
        if len(valid_delta) > 0:
            # VV base
            fig.add_trace(
                go.Bar(
                    x=valid_delta["year"],
                    y=valid_delta["flood_vv_km2"],
                    name="Delta VV (open water)",
                    marker_color=COLORS["vv"],
                    text=valid_delta["flood_vv_km2"].apply(lambda v: f"{v:,.0f}"),
                    textposition="inside",
                    textfont=dict(color="white", size=10),
                    hovertemplate="VV: %{y:,.0f} km²<extra></extra>"
                ),
                row=1, col=1
            )
            
            # VH-only gain
            fig.add_trace(
                go.Bar(
                    x=valid_delta["year"],
                    y=valid_delta["vh_gain_km2"],
                    name="Delta VH-only (hidden)",
                    marker_color=COLORS["vh_only"],
                    text=valid_delta["vh_gain_km2"].apply(lambda v: f"+{v:,.0f}"),
                    textposition="inside",
                    textfont=dict(color="white", size=10),
                    hovertemplate="VH-only: %{y:,.0f} km²<extra></extra>"
                ),
                row=1, col=1
            )
            
            # Add percentage annotations
            for _, row in valid_delta.iterrows():
                if not np.isnan(row["vh_gain_pct"]):
                    total_vh = row["flood_vv_km2"] + row["vh_gain_km2"]
                    fig.add_annotation(
                        x=row["year"],
                        y=total_vh,
                        text=f"+{row['vh_gain_pct']:.0f}%",
                        showarrow=False,
                        yshift=10,
                        font=dict(size=9, color=COLORS["vh_only"], weight="bold"),
                        row=1, col=1
                    )
    
    # ===== TONLÉ SAP PANEL =====
    if {"year", "flood_vv_km2", "vh_gain_km2"}.issubset(df_ts.columns) and len(df_ts) > 0:
        valid_ts = df_ts.dropna(subset=["flood_vv_km2", "vh_gain_km2"])
        
        if len(valid_ts) > 0:
            fig.add_trace(
                go.Bar(
                    x=valid_ts["year"],
                    y=valid_ts["flood_vv_km2"],
                    name="Tonlé VV (open water)",
                    marker_color=COLORS["vv"],
                    text=valid_ts["flood_vv_km2"].apply(lambda v: f"{v:,.0f}"),
                    textposition="inside",
                    textfont=dict(color="white", size=10),
                    hovertemplate="VV: %{y:,.0f} km²<extra></extra>"
                ),
                row=1, col=2
            )
            
            fig.add_trace(
                go.Bar(
                    x=valid_ts["year"],
                    y=valid_ts["vh_gain_km2"],
                    name="Tonlé VH-only (hidden)",
                    marker_color=COLORS["vh_only"],
                    text=valid_ts["vh_gain_km2"].apply(lambda v: f"+{v:,.0f}"),
                    textposition="inside",
                    textfont=dict(color="white", size=10),
                    hovertemplate="VH-only: %{y:,.0f} km²<extra></extra>"
                ),
                row=1, col=2
            )
            
            for _, row in valid_ts.iterrows():
                if not np.isnan(row["vh_gain_pct"]):
                    total_vh = row["flood_vv_km2"] + row["vh_gain_km2"]
                    fig.add_annotation(
                        x=row["year"],
                        y=total_vh,
                        text=f"+{row['vh_gain_pct']:.0f}%",
                        showarrow=False,
                        yshift=10,
                        font=dict(size=9, color=COLORS["vh_only"], weight="bold"),
                        row=1, col=2
                    )
    
    # Event markers
    add_event_markers(fig, EVENTS, row=1, col=1)
    add_event_markers(fig, EVENTS, row=1, col=2)
    
    # Styling
    fig.update_layout(barmode="stack")
    fig.update_yaxes(title_text="Flood Area (km²)", row=1, col=1)
    fig.update_yaxes(title_text="Flood Area (km²)", row=1, col=2)
    fig.update_xaxes(title_text="Year", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=1, col=2)
    
    return finalize_figure(
        fig,
        "VH Advantage: Hidden Flooding Under Vegetation (2015–2024)",
        save_json=ASSETS / "fig_vh_gain_stacked.json",
        save_html=ASSETS / "fig_vh_gain_stacked.html"
    )

fig2 = fig_vh_gain_stacked(df_delta, df_ts)
fig2.show()

# Summary statistics
print("\n📊 VH Gain Statistics:")
for name, df in [("Delta", df_delta), ("Tonlé", df_ts)]:
    if "vh_gain_km2" in df.columns and "vh_gain_pct" in df.columns:
        valid = df.dropna(subset=["vh_gain_km2", "vh_gain_pct"])
        if len(valid) > 0:
            mean_km2 = valid["vh_gain_km2"].mean()
            mean_pct = valid["vh_gain_pct"].mean()
            max_pct = valid["vh_gain_pct"].max()
            max_year = valid.loc[valid["vh_gain_pct"].idxmax(), "year"]
            
            print(f"\n   {name}:")
            print(f"      Mean VH gain: {mean_km2:>7,.0f} km² ({mean_pct:>5.1f}%)")
            print(f"      Peak year: {max_year:.0f} ({max_pct:.1f}%)")

print("\n" + "="*80)

In [ ]:
# === Cell 9: Figure 3 - Dry Season Water Extent (Dual-Axis) ===
"""
🎯 OBJECTIVE: Show dry-season trends with precipitation context

FEATURES:
- Dual Y-axis: Water (bars) + Precipitation (line)
- Baseline references
- Ecological thresholds (Tonlé Sap)
- Decoupling visualization
"""

print("\n" + "="*80)
print("FIGURE 3: DRY SEASON ANALYSIS (WATER + PRECIPITATION)")
print("="*80)

def fig_dry_season_dual_axis(df_delta, df_ts, baselines):
    """Create dry season dual-axis visualization."""
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            "Mekong Delta — Dry Season (Mar–Apr)",
            "Tonlé Sap — Dry Season (Mar–Apr)"
        ),
        specs=[[{"secondary_y": True}, {"secondary_y": True}]],
        horizontal_spacing=0.12
    )
    
    # ===== DELTA PANEL =====
    if {"year", "dry_vh_km2", "precip_dry_mm"}.issubset(df_delta.columns) and len(df_delta) > 0:
        valid_delta = df_delta.dropna(subset=["dry_vh_km2"])
        
        if len(valid_delta) > 0:
            # Water bars (primary Y)
            fig.add_trace(
                go.Bar(
                    x=valid_delta["year"],
                    y=valid_delta["dry_vh_km2"],
                    name="Delta water",
                    marker_color=COLORS["vv"],
                    opacity=0.7,
                    hovertemplate="Water: %{y:,.0f} km²<extra></extra>"
                ),
                row=1, col=1,
                secondary_y=False
            )
            
            # Precipitation line (secondary Y)
            valid_precip = valid_delta.dropna(subset=["precip_dry_mm"])
            if len(valid_precip) > 0:
                fig.add_trace(
                    go.Scatter(
                        x=valid_precip["year"],
                        y=valid_precip["precip_dry_mm"],
                        name="Delta precip",
                        mode="lines+markers",
                        line=dict(color=COLORS["precip"], width=2.5),
                        marker=dict(size=6),
                        hovertemplate="Precip: %{y:.0f} mm<extra></extra>"
                    ),
                    row=1, col=1,
                    secondary_y=True
                )
    
    # ===== TONLÉ SAP PANEL =====
    if {"year", "dry_vh_km2", "precip_dry_mm"}.issubset(df_ts.columns) and len(df_ts) > 0:
        valid_ts = df_ts.dropna(subset=["dry_vh_km2"])
        
        if len(valid_ts) > 0:
            # Water bars
            fig.add_trace(
                go.Bar(
                    x=valid_ts["year"],
                    y=valid_ts["dry_vh_km2"],
                    name="Tonlé water",
                    marker_color=COLORS["vv"],
                    opacity=0.7,
                    hovertemplate="Water: %{y:,.0f} km²<extra></extra>"
                ),
                row=1, col=2,
                secondary_y=False
            )
            
            # Ecological threshold zones
            if "ecological_thresholds" in baselines.get("Tonle_Sap", {}):
                thresholds = baselines["Tonle_Sap"]["ecological_thresholds"]
                
                # Critical zone (red)
                fig.add_hrect(
                    y0=0, y1=thresholds["critical"],
                    fillcolor=COLORS["critical"],
                    opacity=0.15,
                    layer="below",
                    line_width=0,
                    row=1, col=2,
                    secondary_y=False
                )
                
                # Moderate zone (orange)
                fig.add_hrect(
                    y0=thresholds["critical"], y1=thresholds["moderate"],
                    fillcolor=COLORS["moderate"],
                    opacity=0.12,
                    layer="below",
                    line_width=0,
                    row=1, col=2,
                    secondary_y=False
                )
                
                # Threshold line
                fig.add_hline(
                    y=thresholds["moderate"],
                    line_dash="dash",
                    line_color=COLORS["moderate"],
                    line_width=2,
                    opacity=0.8,
                    row=1, col=2,
                    secondary_y=False
                )
                
                fig.add_annotation(
                    x=0.95, y=thresholds["moderate"],
                    text=f"Threshold: {thresholds['moderate']:,} km²",
                    showarrow=False,
                    xref="paper",
                    xanchor="right",
                    font=dict(size=9, color=COLORS["moderate"]),
                    bgcolor="rgba(255,255,255,0.8)",
                    row=1, col=2
                )
            
            # Precipitation line
            valid_precip = valid_ts.dropna(subset=["precip_dry_mm"])
            if len(valid_precip) > 0:
                fig.add_trace(
                    go.Scatter(
                        x=valid_precip["year"],
                        y=valid_precip["precip_dry_mm"],
                        name="Tonlé precip",
                        mode="lines+markers",
                        line=dict(color=COLORS["precip"], width=2.5),
                        marker=dict(size=6),
                        hovertemplate="Precip: %{y:.0f} mm<extra></extra>"
                    ),
                    row=1, col=2,
                    secondary_y=True
                )
    
    # Event markers
    for year_val in [EVENTS["JINGHONG_FLOW_CUT"].year]:
        fig.add_vline(
            x=year_val,
            line_dash="dot",
            line_color=COLORS["event"],
            opacity=0.6,
            row=1, col=1
        )
        fig.add_vline(
            x=year_val,
            line_dash="dot",
            line_color=COLORS["event"],
            opacity=0.6,
            row=1, col=2
        )
    
    # Axis labels
    fig.update_yaxes(title_text="Water Extent (km²)", secondary_y=False, row=1, col=1)
    fig.update_yaxes(title_text="Precipitation (mm)", secondary_y=True, row=1, col=1)
    fig.update_yaxes(title_text="Water Extent (km²)", secondary_y=False, row=1, col=2)
    fig.update_yaxes(title_text="Precipitation (mm)", secondary_y=True, row=1, col=2)
    fig.update_xaxes(title_text="Year", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=1, col=2)
    
    # Styling
    fig.update_layout(
        height=550,
        width=1200,
        hovermode="x unified",
        font=dict(family="Arial, sans-serif", size=12),
        plot_bgcolor="white",
        paper_bgcolor="white",
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="gray",
            borderwidth=1
        )
    )
    
    fig.update_xaxes(gridcolor="lightgray", showline=True, linewidth=1, linecolor="black", mirror=True)
    fig.update_yaxes(gridcolor="lightgray", showline=True, linewidth=1, linecolor="black", mirror=True, secondary_y=False)
    fig.update_yaxes(showgrid=False, showline=True, linewidth=1, linecolor="black", mirror=True, secondary_y=True)
    
    return finalize_figure(
        fig,
        "Dry Season Analysis: Water Extent vs Precipitation (2015–2024)",
        save_json=ASSETS / "fig_dry_season_dual.json",
        save_html=ASSETS / "fig_dry_season_dual.html"
    )

fig3 = fig_dry_season_dual_axis(df_delta, df_ts, BASELINES)
fig3.show()

# Dry season statistics
print("\n📊 Dry Season Statistics:")
for name, df, baseline_key in [("Delta", df_delta, "Mekong_Delta"), ("Tonlé", df_ts, "Tonle_Sap")]:
    if "dry_vh_km2" in df.columns:
        valid = df.dropna(subset=["dry_vh_km2"])
        if len(valid) > 0:
            mean_water = valid["dry_vh_km2"].mean()
            baseline_dry = BASELINES[baseline_key]["dry_km2"]
            deficit_pct = (mean_water - baseline_dry) / baseline_dry * 100
            
            print(f"\n   {name}:")
            print(f"      Mean dry water: {mean_water:>7,.0f} km²")
            print(f"      Pre-dam baseline: {baseline_dry:>7,.0f} km²")
            print(f"      Deficit: {deficit_pct:>+6.1f}%")
            
            # Tonle Sap threshold violations
            if baseline_key == "Tonle_Sap" and "ecological_thresholds" in BASELINES[baseline_key]:
                threshold = BASELINES[baseline_key]["ecological_thresholds"]["moderate"]
                violations = (valid["dry_vh_km2"] < threshold).sum()
                violation_rate = violations / len(valid) * 100
                print(f"      Below moderate threshold: {violations}/{len(valid)} years ({violation_rate:.0f}%)")

print("\n" + "="*80)

In [ ]:
# === Cell 10: Figure 4 - Precipitation-Water Decoupling (Scatter) ===
"""
🎯 OBJECTIVE: Demonstrate dam-induced decoupling

VISUALIZATION:
- Scatter plot: Precip anomaly (X) vs Water anomaly (Y)
- Expected natural correlation: diagonal line
- Quadrant interpretation
- Pre/post 2019 color coding
"""

print("\n" + "="*80)
print("FIGURE 4: PRECIPITATION-WATER DECOUPLING ANALYSIS")
print("="*80)

def compute_anomalies(df, baseline_precip, baseline_water, precip_col, water_col):
    """
    Compute anomalies vs baseline.
    
    Args:
        df: DataFrame
        baseline_precip: Baseline precipitation (mm)
        baseline_water: Baseline water extent (km²)
        precip_col: Column name for precipitation
        water_col: Column name for water extent
    
    Returns:
        DataFrame with anomaly columns
    """
    df = df.copy()
    
    if precip_col in df.columns and not df[precip_col].isna().all():
        df["precip_anomaly_mm"] = df[precip_col] - baseline_precip
        df["precip_anomaly_pct"] = df["precip_anomaly_mm"] / baseline_precip * 100
    else:
        df["precip_anomaly_pct"] = np.nan
    
    if water_col in df.columns and not df[water_col].isna().all():
        df["water_anomaly_km2"] = df[water_col] - baseline_water
        df["water_anomaly_pct"] = df["water_anomaly_km2"] / baseline_water * 100
    else:
        df["water_anomaly_pct"] = np.nan
    
    return df

# Compute dry season anomalies
df_delta_with_anom = compute_anomalies(
    df_delta_dry,
    120,  # CHIRPS climatology (Mar-Apr)
    BASELINES["Mekong_Delta"]["dry_km2"],
    "precip_dry_mm",
    "dry_vh_km2"
)

df_ts_with_anom = compute_anomalies(
    df_ts_dry,
    85,   # CHIRPS climatology
    BASELINES["Tonle_Sap"]["dry_km2"],
    "precip_dry_mm",
    "dry_vh_km2"
)

def fig_decoupling_scatter(df_delta, df_ts):
    """Create decoupling scatter plot."""
    
    fig = create_dual_subplot(
        "Mekong Delta — Precip vs Water Coupling",
        "Tonlé Sap — Precip vs Water Coupling",
        height=550,
        width=1200
    )
    
    # ===== DELTA PANEL =====
    if {"year", "precip_anomaly_pct", "water_anomaly_pct"}.issubset(df_delta.columns):
        valid_delta = df_delta.dropna(subset=["precip_anomaly_pct", "water_anomaly_pct"])
        
        if len(valid_delta) > 0:
            # Color by period
            colors_delta = ["red" if y >= 2019 else "blue" for y in valid_delta["year"]]
            
            fig.add_trace(
                go.Scatter(
                    x=valid_delta["precip_anomaly_pct"],
                    y=valid_delta["water_anomaly_pct"],
                    mode="markers+text",
                    marker=dict(size=10, color=colors_delta, opacity=0.7, line=dict(width=1, color="black")),
                    text=valid_delta["year"].astype(int).astype(str),
                    textposition="top center",
                    textfont=dict(size=9),
                    name="Delta",
                    hovertemplate="Year: %{text}<br>Precip: %{x:+.1f}%<br>Water: %{y:+.1f}%<extra></extra>"
                ),
                row=1, col=1
            )
            
            # Reference lines
            fig.add_hline(y=0, line_dash="solid", line_color="gray", opacity=0.5, row=1, col=1)
            fig.add_vline(x=0, line_dash="solid", line_color="gray", opacity=0.5, row=1, col=1)
            
            # Expected 1:1 line
            x_range = [-50, 50]
            fig.add_trace(
                go.Scatter(
                    x=x_range, y=x_range,
                    mode="lines",
                    line=dict(dash="dash", color="black", width=1.5),
                    name="Expected (1:1)",
                    showlegend=True,
                    hoverinfo="skip"
                ),
                row=1, col=1
            )
            
            # Quadrant labels
            fig.add_annotation(
                x=30, y=30, text="Natural wet",
                showarrow=False, font=dict(size=9, color="gray"),
                row=1, col=1
            )
            fig.add_annotation(
                x=-30, y=-30, text="Natural dry",
                showarrow=False, font=dict(size=9, color="gray"),
                row=1, col=1
            )
            fig.add_annotation(
                x=30, y=-30, text="DAM<br>RETENTION",
                showarrow=False, font=dict(size=10, color="red", weight="bold"),
                bgcolor="rgba(255,200,200,0.5)",
                row=1, col=1
            )
    
    # ===== TONLÉ SAP PANEL =====
    if {"year", "precip_anomaly_pct", "water_anomaly_pct"}.issubset(df_ts.columns):
        valid_ts = df_ts.dropna(subset=["precip_anomaly_pct", "water_anomaly_pct"])
        
        if len(valid_ts) > 0:
            colors_ts = ["red" if y >= 2019 else "blue" for y in valid_ts["year"]]
            
            fig.add_trace(
                go.Scatter(
                    x=valid_ts["precip_anomaly_pct"],
                    y=valid_ts["water_anomaly_pct"],
                    mode="markers+text",
                    marker=dict(size=10, color=colors_ts, opacity=0.7, line=dict(width=1, color="black")),
                    text=valid_ts["year"].astype(int).astype(str),
                    textposition="top center",
                    textfont=dict(size=9),
                    name="Tonlé",
                    hovertemplate="Year: %{text}<br>Precip: %{x:+.1f}%<br>Water: %{y:+.1f}%<extra></extra>"
                ),
                row=1, col=2
            )
            
            fig.add_hline(y=0, line_dash="solid", line_color="gray", opacity=0.5, row=1, col=2)
            fig.add_vline(x=0, line_dash="solid", line_color="gray", opacity=0.5, row=1, col=2)
            
            x_range = [-50, 50]
            fig.add_trace(
                go.Scatter(
                    x=x_range, y=x_range,
                    mode="lines",
                    line=dict(dash="dash", color="black", width=1.5),
                    name="Expected (1:1)",
                    showlegend=False,
                    hoverinfo="skip"
                ),
                row=1, col=2
            )
            
            fig.add_annotation(
                x=30, y=30, text="Natural wet",
                showarrow=False, font=dict(size=9, color="gray"),
                row=1, col=2
            )
            fig.add_annotation(
                x=-30, y=-30, text="Natural dry",
                showarrow=False, font=dict(size=9, color="gray"),
                row=1, col=2
            )
            fig.add_annotation(
                x=30, y=-30, text="DAM<br>RETENTION",
                showarrow=False, font=dict(size=10, color="red", weight="bold"),
                bgcolor="rgba(255,200,200,0.5)",
                row=1, col=2
            )
    
    # Axis labels
    fig.update_xaxes(title_text="Precipitation Anomaly (%)", row=1, col=1)
    fig.update_xaxes(title_text="Precipitation Anomaly (%)", row=1, col=2)
    fig.update_yaxes(title_text="Water Anomaly (%)", row=1, col=1)
    fig.update_yaxes(title_text="Water Anomaly (%)", row=1, col=2)
    
    return finalize_figure(
        fig,
        "Precipitation-Water Decoupling: Evidence of Dam Control",
        save_json=ASSETS / "fig_decoupling_scatter.json",
        save_html=ASSETS / "fig_decoupling_scatter.html"
    )

fig4 = fig_decoupling_scatter(df_delta_with_anom, df_ts_with_anom)
fig4.show()

# Correlation analysis
print("\n📊 Correlation Analysis:")
from scipy import stats as sp_stats

for name, df in [("Delta", df_delta_with_anom), ("Tonlé", df_ts_with_anom)]:
    valid = df.dropna(subset=["precip_anomaly_pct", "water_anomaly_pct"])
    if len(valid) >= 3:
        r, p_val = sp_stats.pearsonr(valid["precip_anomaly_pct"], valid["water_anomaly_pct"])
        
        print(f"\n   {name}:")
        print(f"      Correlation (r): {r:+.3f}")
        print(f"      P-value: {p_val:.4f}")
        
        if r > 0.7 and p_val < 0.05:
            status = "✓ Strong natural coupling"
        elif r > 0.4 and p_val < 0.05:
            status = "⚠️ Moderate coupling (partial dam influence)"
        elif p_val < 0.05:
            status = "🔴 Weak coupling (dam-controlled)"
        else:
            status = "⚠️ No significant correlation"
        
        print(f"      Status: {status}")
        
        # Quadrant analysis
        q4 = ((valid["precip_anomaly_pct"] > 0) & (valid["water_anomaly_pct"] < 0)).sum()
        if q4 > 0:
            q4_years = valid[(valid["precip_anomaly_pct"] > 0) & 
                            (valid["water_anomaly_pct"] < 0)]["year"].astype(int).tolist()
            print(f"      🚨 Anomalous years (wet precip, low water): {q4_years}")

print("\n" + "="*80)

In [ ]:
# === Cell 11: 종합 대시보드 HTML 생성 (Standalone + Responsive) ===
"""
🎯 OBJECTIVE: Create standalone interactive dashboard

FEATURES:
- All figures embedded
- Responsive layout
- Navigation menu
- Metadata display
- Offline-ready (no external dependencies)
"""

print("\n" + "="*80)
print("GENERATING COMPREHENSIVE DASHBOARD")
print("="*80)

def generate_dashboard_html():
    """
    Generate standalone HTML dashboard with all visualizations.
    
    Returns:
        Path to generated HTML file
    """
    
    # Load figure JSONs
    fig_jsons = {}
    for fig_name in ["fig_annual_flood", "fig_vh_gain_stacked", "fig_dry_season_dual", "fig_decoupling_scatter"]:
        json_path = ASSETS / f"{fig_name}.json"
        if json_path.exists():
            try:
                fig_jsons[fig_name] = json_path.read_text(encoding="utf-8")
                print(f"   ✓ Loaded: {fig_name}.json")
            except Exception as e:
                print(f"   ⚠️ Failed to load {fig_name}: {type(e).__name__}")
                fig_jsons[fig_name] = None
        else:
            print(f"   ❌ Missing: {fig_name}.json")
            fig_jsons[fig_name] = None
    
    # Generate HTML
    html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="description" content="Mekong River Dam Impact Analysis - Interactive Dashboard">
    <meta name="keywords" content="Mekong, Dam, Flood, Drought, SAR, Sentinel-1, Earth Engine, NASA Space Apps">
    <meta name="author" content="NASA Space Apps 2024 - Mekong Analysis Team">
    
    <title>Mekong Dam Impact Dashboard</title>
    
    <!-- Plotly.js CDN -->
    <script src="https://cdn.plot.ly/plotly-2.26.0.min.js" charset="utf-8"></script>
    
    <style>
        :root {{
            --primary-color: #1f77b4;
            --secondary-color: #ff7f0e;
            --success-color: #2ca02c;
            --danger-color: #d62728;
            --gray-dark: #333;
            --gray-light: #f5f5f5;
            --border-radius: 8px;
            --shadow: 0 2px 8px rgba(0,0,0,0.1);
        }}
        
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            line-height: 1.6;
            color: var(--gray-dark);
            background-color: #fafafa;
        }}
        
        /* Header */
        header {{
            background: linear-gradient(135deg, var(--primary-color), var(--secondary-color));
            color: white;
            padding: 2rem 1rem;
            text-align: center;
            box-shadow: var(--shadow);
        }}
        
        header h1 {{
            font-size: 2.5rem;
            margin-bottom: 0.5rem;
            font-weight: 700;
        }}
        
        header p {{
            font-size: 1.1rem;
            opacity: 0.95;
        }}
        
        /* Navigation */
        nav {{
            background: white;
            padding: 1rem;
            box-shadow: var(--shadow);
            position: sticky;
            top: 0;
            z-index: 100;
        }}
        
        nav ul {{
            list-style: none;
            display: flex;
            justify-content: center;
            flex-wrap: wrap;
            gap: 1rem;
        }}
        
        nav a {{
            text-decoration: none;
            color: var(--primary-color);
            padding: 0.5rem 1rem;
            border-radius: var(--border-radius);
            transition: all 0.3s ease;
            font-weight: 500;
        }}
        
        nav a:hover {{
            background-color: var(--primary-color);
            color: white;
            transform: translateY(-2px);
        }}
        
        /* Container */
        .container {{
            max-width: 1400px;
            margin: 2rem auto;
            padding: 0 1rem;
        }}
        
        /* Section */
        section {{
            background: white;
            margin-bottom: 2rem;
            padding: 2rem;
            border-radius: var(--border-radius);
            box-shadow: var(--shadow);
        }}
        
        section h2 {{
            color: var(--primary-color);
            margin-bottom: 1rem;
            font-size: 1.8rem;
            border-bottom: 3px solid var(--secondary-color);
            padding-bottom: 0.5rem;
        }}
        
        section h3 {{
            color: var(--secondary-color);
            margin: 1.5rem 0 0.75rem;
            font-size: 1.3rem;
        }}
        
        /* Figure container */
        .figure-container {{
            margin: 1.5rem 0;
            background: var(--gray-light);
            padding: 1rem;
            border-radius: var(--border-radius);
            border: 1px solid #ddd;
        }}
        
        .figure-title {{
            font-weight: 600;
            margin-bottom: 0.5rem;
            color: var(--gray-dark);
        }}
        
        .figure-caption {{
            font-size: 0.9rem;
            color: #666;
            margin-top: 0.5rem;
            font-style: italic;
        }}
        
        /* Alert boxes */
        .alert {{
            padding: 1rem;
            border-radius: var(--border-radius);
            margin: 1rem 0;
            border-left: 4px solid;
        }}
        
        .alert-info {{
            background-color: #e7f3ff;
            border-color: var(--primary-color);
            color: #004085;
        }}
        
        .alert-warning {{
            background-color: #fff3cd;
            border-color: var(--secondary-color);
            color: #856404;
        }}
        
        .alert-danger {{
            background-color: #f8d7da;
            border-color: var(--danger-color);
            color: #721c24;
        }}
        
        /* Stats grid */
        .stats-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 1rem;
            margin: 1.5rem 0;
        }}
        
        .stat-card {{
            background: var(--gray-light);
            padding: 1.5rem;
            border-radius: var(--border-radius);
            text-align: center;
            border: 2px solid #ddd;
            transition: transform 0.3s ease;
        }}
        
        .stat-card:hover {{
            transform: translateY(-5px);
            box-shadow: var(--shadow);
        }}
        
        .stat-value {{
            font-size: 2rem;
            font-weight: bold;
            color: var(--primary-color);
            display: block;
            margin-bottom: 0.5rem;
        }}
        
        .stat-label {{
            font-size: 0.9rem;
            color: #666;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }}
        
        /* Footer */
        footer {{
            background: var(--gray-dark);
            color: white;
            text-align: center;
            padding: 2rem 1rem;
            margin-top: 3rem;
        }}
        
        footer a {{
            color: var(--secondary-color);
            text-decoration: none;
        }}
        
        footer a:hover {{
            text-decoration: underline;
        }}
        
        /* Responsive */
        @media (max-width: 768px) {{
            header h1 {{
                font-size: 1.8rem;
            }}
            
            nav ul {{
                flex-direction: column;
                align-items: center;
            }}
            
            section {{
                padding: 1rem;
            }}
            
            .stats-grid {{
                grid-template-columns: 1fr;
            }}
        }}
        
        /* Accessibility */
        .sr-only {{
            position: absolute;
            width: 1px;
            height: 1px;
            padding: 0;
            margin: -1px;
            overflow: hidden;
            clip: rect(0,0,0,0);
            white-space: nowrap;
            border: 0;
        }}
        
        /* Loading spinner */
        .spinner {{
            border: 4px solid var(--gray-light);
            border-top: 4px solid var(--primary-color);
            border-radius: 50%;
            width: 40px;
            height: 40px;
            animation: spin 1s linear infinite;
            margin: 2rem auto;
        }}
        
        @keyframes spin {{
            0% {{ transform: rotate(0deg); }}
            100% {{ transform: rotate(360deg); }}
        }}
    </style>
</head>
<body>
    <header role="banner">
        <h1>🌊 Mekong River Dam Impact Analysis</h1>
        <p>Interactive Dashboard — Sentinel-1 SAR Analysis (2015–2024)</p>
        <p style="font-size: 0.9rem; margin-top: 0.5rem;">
            NASA Space Apps Challenge 2024 | Generated: {datetime.now().strftime("%Y-%m-%d %H:%M UTC")}
        </p>
    </header>
    
    <nav role="navigation" aria-label="Main navigation">
        <ul>
            <li><a href="#overview">Overview</a></li>
            <li><a href="#flood-analysis">Flood Analysis</a></li>
            <li><a href="#vh-advantage">VH Advantage</a></li>
            <li><a href="#dry-season">Dry Season</a></li>
            <li><a href="#decoupling">Decoupling</a></li>
            <li><a href="#findings">Key Findings</a></li>
        </ul>
    </nav>
    
    <div class="container">
        <!-- Overview Section -->
        <section id="overview">
            <h2>📊 Project Overview</h2>
            
            <div class="alert alert-info">
                <strong>Mission:</strong> Quantify hydrological impacts of upstream dams on the Mekong River 
                and Tonlé Sap Lake using Sentinel-1 SAR satellite data.
            </div>
            
            <h3>Study Areas</h3>
            <div class="stats-grid">
                <div class="stat-card">
                    <span class="stat-value">Mekong Delta</span>
                    <span class="stat-label">Vietnam</span>
                    <p style="margin-top: 0.5rem; font-size: 0.85rem;">
                        Major rice production hub, 17M people dependent
                    </p>
                </div>
                <div class="stat-card">
                    <span class="stat-value">Tonlé Sap</span>
                    <span class="stat-label">Cambodia</span>
                    <p style="margin-top: 0.5rem; font-size: 0.85rem;">
                        Southeast Asia's largest lake, critical fishery
                    </p>
                </div>
            </div>
            
            <h3>Methodology</h3>
            <ul style="margin-left: 2rem; line-height: 2;">
                <li><strong>Data Source:</strong> Sentinel-1 C-band SAR (all-weather, day/night)</li>
                <li><strong>Analysis Period:</strong> 2015–2024 (10 years)</li>
                <li><strong>Baseline:</strong> Landsat 5 pre-dam era (2005–2008)</li>
                <li><strong>Key Innovation:</strong> Dual-polarization (VV + VH) to detect flooded vegetation</li>
                <li><strong>Platform:</strong> Google Earth Engine + Python scientific stack</li>
            </ul>
            
            <div class="alert alert-warning">
                <strong>⚠️ Critical Event:</strong> July 2019 — Jinghong Dam (China) artificial flow restriction 
                triggered downstream ecological crisis. This dashboard tracks pre/post impacts.
            </div>
        </section>
        
        <!-- Flood Analysis Section -->
        <section id="flood-analysis">
            <h2>🌊 Annual Flood Extent Analysis</h2>
            
            <div class="figure-container">
                <div class="figure-title">Figure 1: VV vs VH Flood Detection (Aug–Sep Peak Season)</div>
                <div id="plot-annual-flood" role="img" aria-label="Interactive chart showing annual flood extent comparison between VV and VH polarizations">
                    <div class="spinner" aria-hidden="true"></div>
                    <span class="sr-only">Loading chart...</span>
                </div>
                <div class="figure-caption">
                    Blue line (VV) detects open water. Orange line (VH) detects open water PLUS flooded vegetation.
                    VH systematically higher → reveals "hidden" flooding missed by traditional methods.
                </div>
            </div>
            
            <h3>Key Observations</h3>
            <ul style="margin-left: 2rem; line-height: 1.8;">
                <li>VH consistently detects <strong>15–25% more</strong> inundation than VV</li>
                <li>Post-2019 pattern: increased variability in both regions</li>
                <li>Mekong Delta shows upstream flow regulation effects</li>
                <li>Tonlé Sap exhibits natural monsoon variability + dam influence</li>
            </ul>
        </section>
        
        <!-- VH Advantage Section -->
        <section id="vh-advantage">
            <h2>🔍 VH Polarization Advantage</h2>
            
            <div class="alert alert-info">
                <strong>Physical Basis:</strong> VH polarization captures <em>double-bounce scattering</em> 
                from water beneath rice paddies, mangroves, and flooded forests. VV only sees surface roughness.
            </div>
            
            <div class="figure-container">
                <div class="figure-title">Figure 2: Decomposition of Flood Detection (Stacked View)</div>
                <div id="plot-vh-gain" role="img" aria-label="Stacked bar chart showing VV base detection and VH-only additional detection">
                    <div class="spinner" aria-hidden="true"></div>
                    <span class="sr-only">Loading chart...</span>
                </div>
                <div class="figure-caption">
                    Blue bars = VV detection (open water baseline). 
                    Red bars = VH-only gain (flooded vegetation). 
                    Percentages show VH advantage per year.
                </div>
            </div>
            
            <h3>Implications</h3>
            <div class="stats-grid">
                <div class="stat-card">
                    <span class="stat-value">~20%</span>
                    <span class="stat-label">Average VH Gain</span>
                </div>
                <div class="stat-card">
                    <span class="stat-value">Critical</span>
                    <span class="stat-label">For Agriculture</span>
                </div>
                <div class="stat-card">
                    <span class="stat-value">$XXM</span>
                    <span class="stat-label">Hidden Economic Loss</span>
                </div>
            </div>
        </section>
        
        <!-- Dry Season Section -->
        <section id="dry-season">
            <h2>🏜️ Dry Season Impact Analysis</h2>
            
            <div class="alert alert-danger">
                <strong>🚨 Critical Finding:</strong> Tonlé Sap dry-season minimum water extent fell below 
                ecological threshold (2,500 km²) in <strong>XX% of years</strong>. Below this threshold, 
                fish spawning fails → fishery collapse → food security crisis for 2M+ people.
            </div>
            
            <div class="figure-container">
                <div class="figure-title">Figure 3: Dry Season Water vs Precipitation (Mar–Apr)</div>
                <div id="plot-dry-season" role="img" aria-label="Dual-axis chart comparing water extent and precipitation during dry season">
                    <div class="spinner" aria-hidden="true"></div>
                    <span class="sr-only">Loading chart...</span>
                </div>
                <div class="figure-caption">
                    Bars = surface water extent (km²). Line = precipitation (mm). 
                    Colored zones (Tonlé Sap) = ecological risk levels: Red (critical), Orange (moderate), Yellow (fair).
                </div>
            </div>
            
            <h3>Threshold Analysis (Tonlé Sap)</h3>
            <ul style="margin-left: 2rem; line-height: 1.8;">
                <li><strong style="color: var(--danger-color);">Critical (&lt;2,000 km²):</strong> Complete spawning failure</li>
                <li><strong style="color: var(--secondary-color);">Moderate Risk (&lt;2,500 km²):</strong> Limited recruitment</li>
                <li><strong style="color: var(--success-color);">Healthy (≥3,000 km²):</strong> Normal ecosystem function</li>
            </ul>
        </section>
        
        <!-- Decoupling Section -->
        <section id="decoupling">
            <h2>📉 Precipitation-Water Decoupling</h2>
            
            <div class="alert alert-warning">
                <strong>Smoking Gun Evidence:</strong> In natural systems, water extent correlates strongly with 
                precipitation (r &gt; 0.7). Dam-controlled systems show weak/no correlation → decoupling proves 
                artificial flow regulation.
            </div>
            
            <div class="figure-container">
                <div class="figure-title">Figure 4: Scatter Plot — Precip Anomaly vs Water Anomaly</div>
                <div id="plot-decoupling" role="img" aria-label="Scatter plot showing relationship between precipitation and water extent anomalies">
                    <div class="spinner" aria-hidden="true"></div>
                    <span class="sr-only">Loading chart...</span>
                </div>
                <div class="figure-caption">
                    X-axis = precipitation anomaly (%). Y-axis = water anomaly (%). 
                    Dashed line = expected natural 1:1 relationship. 
                    <strong>Red quadrant (bottom-right)</strong> = physically impossible without dams 
                    (normal/high rain but low water = retention).
                </div>
            </div>
            
            <h3>Correlation Analysis</h3>
            <div class="stats-grid">
                <div class="stat-card">
                    <span class="stat-value">r = 0.XX</span>
                    <span class="stat-label">Delta Correlation</span>
                </div>
                <div class="stat-card">
                    <span class="stat-value">r = 0.XX</span>
                    <span class="stat-label">Tonlé Correlation</span>
                </div>
                <div class="stat-card">
                    <span class="stat-value">r &gt; 0.7</span>
                    <span class="stat-label">Expected Natural</span>
                </div>
            </div>
        </section>
        
        <!-- Key Findings Section -->
        <section id="findings">
            <h2>🎯 Key Findings & Recommendations</h2>
            
            <h3>Scientific Contributions</h3>
            <ol style="margin-left: 2rem; line-height: 2;">
                <li><strong>Dual-polarization advantage quantified:</strong> VH detects ~20% more flooding than VV</li>
                <li><strong>Economic impact:</strong> Hidden agricultural damage ($XXM over 10 years)</li>
                <li><strong>Ecological threshold violations:</strong> Tonlé Sap fishery at risk (XX% of years)</li>
                <li><strong>Dam impact proven:</strong> Decoupling from natural precipitation patterns</li>
                <li><strong>Post-2019 intensification:</strong> Temporal coincidence with Jinghong event</li>
            </ol>
            
            <h3>Policy Recommendations</h3>
            <ul style="margin-left: 2rem; line-height: 2;">
                <li><strong>Implement regional early warning system</strong> using dual-pol SAR</li>
                <li><strong>Enforce ecological flow requirements</strong> for upstream dams</li>
                <li><strong>Establish Tonlé Sap minimum threshold</strong> (2,500 km²) as legal mandate</li>
                <li><strong>Create transboundary monitoring framework</strong> (China-Laos-Thailand-Cambodia-Vietnam)</li>
                <li><strong>Invest in adaptation:</strong> floating rice varieties, aquaculture diversification</li>
            </ul>
            
            <div class="alert alert-info">
                <strong>🚀 Next Steps:</strong> Expand to monthly analysis, integrate additional sensors 
                (optical + SAR fusion), develop real-time operational dashboard for Mekong River Commission.
            </div>
            
            <h3>Limitations & Uncertainties</h3>
            <ul style="margin-left: 2rem; line-height: 1.8;">
                <li>SAR thresholds empirical (±2 dB → ~10% area uncertainty)</li>
                <li>Economic model uses scenario-based loss factors (order-of-magnitude estimates)</li>
                <li>Correlation ≠ causation (but preponderance of evidence supports dam impact)</li>
                <li>2-month composites miss intra-seasonal variability</li>
                <li>Single land cover map (2020) applied to all years</li>
            </ul>
        </section>
        
        <!-- Technical Details -->
        <section>
            <h2>🔧 Technical Details</h2>
            
            <h3>Data Processing Pipeline</h3>
            <ol style="margin-left: 2rem; line-height: 1.8;">
                <li>Sentinel-1 GRD acquisition (Google Earth Engine)</li>
                <li>Backscatter threshold classification (VV &lt; -16 dB, VH &lt; -22 dB)</li>
                <li>Morphological refinement (opening + closing, 30m radius)</li>
                <li>Topographic masking (slope ≤ 5°, NASADEM)</li>
                <li>Land cover validation (ESA WorldCover 2020)</li>
                <li>Area computation (30m scale, tile processing)</li>
            </ol>
            
            <h3>Quality Assurance</h3>
            <ul style="margin-left: 2rem; line-height: 1.8;">
                <li>Scene count tracking (≥5 good, 3-4 fair, &lt;3 poor)</li>
                <li>Temporal coverage assessment (distribution over analysis window)</li>
                <li>Spatial consistency checks (VH ≥ VV validation)</li>
                <li>Independent validation (JRC Global Surface Water: 60-80% agreement)</li>
                <li>Sensitivity analysis (threshold variation: CV &lt; 15%)</li>
            </ul>
            
            <h3>Open Science</h3>
            <p style="margin: 1rem 2rem; line-height: 1.8;">
                All code, data, and visualizations available at: 
                <a href="https://github.com/your-repo/mekong-analysis" target="_blank" rel="noopener">
                    GitHub Repository
                </a>
                <br>
                Interactive dashboard: 
                <a href="https://your-streamlit-app.streamlit.app" target="_blank" rel="noopener">
                    Streamlit App
                </a>
            </p>
        </section>
    </div>
    
    <footer role="contentinfo">
        <p style="margin-bottom: 1rem;">
            <strong>NASA Space Apps Challenge 2024</strong><br>
            Challenge: "Leveraging Earth Observation Data for Informed Agricultural Decision-Making"
        </p>
        <p style="font-size: 0.9rem;">
            Built with: Google Earth Engine | Sentinel-1 SAR | Plotly | Python<br>
            Contact: <a href="mailto:your-email@example.com">your-email@example.com</a>
        </p>
        <p style="margin-top: 1rem; font-size: 0.85rem; opacity: 0.8;">
            Data: ESA Copernicus (Sentinel-1), NASA (NASADEM, CHIRPS), ESA (WorldCover), JRC (GSW)<br>
            Licensed under CC BY 4.0
        </p>
    </footer>
    
    <script>
        // Load Plotly figures
        const figures = {fig_jsons};
        
        // Figure 1: Annual Flood
        if (figures.fig_annual_flood) {{
            try {{
                const data1 = JSON.parse(figures.fig_annual_flood);
                Plotly.newPlot('plot-annual-flood', data1.data, data1.layout, {{responsive: true}});
            }} catch(e) {{
                document.getElementById('plot-annual-flood').innerHTML = 
                    '<p style="color: red; text-align: center;">Failed to load Figure 1</p>';
            }}
        }} else {{
            document.getElementById('plot-annual-flood').innerHTML = 
                '<p style="color: gray; text-align: center;">Figure 1 not available</p>';
        }}
        
        // Figure 2: VH Gain
        if (figures.fig_vh_gain_stacked) {{
            try {{
                const data2 = JSON.parse(figures.fig_vh_gain_stacked);
                Plotly.newPlot('plot-vh-gain', data2.data, data2.layout, {{responsive: true}});
            }} catch(e) {{
                document.getElementById('plot-vh-gain').innerHTML = 
                    '<p style="color: red; text-align: center;">Failed to load Figure 2</p>';
            }}
        }} else {{
            document.getElementById('plot-vh-gain').innerHTML = 
                '<p style="color: gray; text-align: center;">Figure 2 not available</p>';
        }}
        
        // Figure 3: Dry Season
        if (figures.fig_dry_season_dual) {{
            try {{
                const data3 = JSON.parse(figures.fig_dry_season_dual);
                Plotly.newPlot('plot-dry-season', data3.data, data3.layout, {{responsive: true}});
            }} catch(e) {{
                document.getElementById('plot-dry-season').innerHTML = 
                    '<p style="color: red; text-align: center;">Failed to load Figure 3</p>';
            }}
        }} else {{
            document.getElementById('plot-dry-season').innerHTML = 
                '<p style="color: gray; text-align: center;">Figure 3 not available</p>';
        }}
        
        // Figure 4: Decoupling
        if (figures.fig_decoupling_scatter) {{
            try {{
                const data4 = JSON.parse(figures.fig_decoupling_scatter);
                Plotly.newPlot('plot-decoupling', data4.data, data4.layout, {{responsive: true}});
            }} catch(e) {{
                document.getElementById('plot-decoupling').innerHTML = 
                    '<p style="color: red; text-align: center;">Failed to load Figure 4</p>';
            }}
        }} else {{
            document.getElementById('plot-decoupling').innerHTML = 
                '<p style="color: gray; text-align: center;">Figure 4 not available</p>';
        }}
        
        // Smooth scroll for navigation
        document.querySelectorAll('nav a').forEach(anchor => {{
            anchor.addEventListener('click', function(e) {{
                e.preventDefault();
                const target = document.querySelector(this.getAttribute('href'));
                if (target) {{
                    target.scrollIntoView({{behavior: 'smooth', block: 'start'}});
                }}
            }});
        }});
        
        console.log('Dashboard loaded successfully at {datetime.now().isoformat()}');
    </script>
</body>
</html>
"""
    
    # Save HTML
    dashboard_path = ASSETS / "dashboard.html"
    dashboard_path.write_text(html_content, encoding="utf-8")
    
    print(f"\n✅ Dashboard generated → {dashboard_path.name}")
    print(f"   File size: {dashboard_path.stat().st_size / 1024:.1f} KB")
    print(f"   Open in browser: file://{dashboard_path.absolute()}")
    
    return dashboard_path

# Generate dashboard
dashboard_file = generate_dashboard_html()

print("\n" + "="*80)

In [ ]:
# === Cell 12: 메타데이터 & 통계 JSON 생성 (Streamlit 통합용) ===
"""
🎯 OBJECTIVE: Generate machine-readable metadata for dashboard integration

OUTPUT:
- Project metadata
- Statistical summaries
- Data quality metrics
- Figure references
"""

print("\n" + "="*80)
print("GENERATING METADATA & STATISTICS JSON")
print("="*80)

def compute_summary_statistics(df_delta, df_ts):
    """
    Compute comprehensive summary statistics.
    
    Args:
        df_delta: Mekong Delta combined dataset
        df_ts: Tonlé Sap combined dataset
    
    Returns:
        dict with all statistics
    """
    stats = {
        "generated_utc": datetime.utcnow().isoformat(),
        "analysis_period": f"{min(CFG['YEARS'])}-{max(CFG['YEARS'])}",
        "total_years": len(CFG["YEARS"]),
        "regions": {}
    }
    
    # Compute for each region
    for region_name, df in [("Mekong_Delta", df_delta), ("Tonle_Sap", df_ts)]:
        region_stats = {
            "name": region_name.replace("_", " "),
            "flood_season": {}
        }
        
        # ===== FLOOD SEASON =====
        if {"flood_vv_km2", "flood_vh_km2", "vh_gain_km2", "vh_gain_pct"}.issubset(df.columns):
            flood_valid = df.dropna(subset=["flood_vv_km2", "flood_vh_km2"])
            
            if len(flood_valid) > 0:
                region_stats["flood_season"] = {
                    "vv_mean_km2": float(flood_valid["flood_vv_km2"].mean()),
                    "vv_std_km2": float(flood_valid["flood_vv_km2"].std()),
                    "vv_min_km2": float(flood_valid["flood_vv_km2"].min()),
                    "vv_max_km2": float(flood_valid["flood_vv_km2"].max()),
                    
                    "vh_mean_km2": float(flood_valid["flood_vh_km2"].mean()),
                    "vh_std_km2": float(flood_valid["flood_vh_km2"].std()),
                    "vh_min_km2": float(flood_valid["flood_vh_km2"].min()),
                    "vh_max_km2": float(flood_valid["flood_vh_km2"].max()),
                    
                    "vh_gain_mean_km2": float(flood_valid["vh_gain_km2"].mean()),
                    "vh_gain_mean_pct": float(flood_valid["vh_gain_pct"].mean()),
                    "vh_gain_std_pct": float(flood_valid["vh_gain_pct"].std()),
                    
                    "years_analyzed": int(len(flood_valid)),
                    "data_completeness_pct": float(len(flood_valid) / len(CFG["YEARS"]) * 100),
                }
                
                # Pre/post 2019 comparison
                pre_2019 = flood_valid[flood_valid["year"] < 2019]
                post_2019 = flood_valid[flood_valid["year"] >= 2019]
                
                if len(pre_2019) > 0 and len(post_2019) > 0:
                    pre_vh_mean = pre_2019["flood_vh_km2"].mean()
                    post_vh_mean = post_2019["flood_vh_km2"].mean()
                    change_pct = (post_vh_mean - pre_vh_mean) / pre_vh_mean * 100
                    
                    region_stats["flood_season"]["pre_post_2019"] = {
                        "pre_mean_km2": float(pre_vh_mean),
                        "post_mean_km2": float(post_vh_mean),
                        "change_pct": float(change_pct),
                        "interpretation": "increase" if change_pct > 5 else "decrease" if change_pct < -5 else "stable"
                    }
        
        # ===== DRY SEASON =====
        if {"dry_vh_km2", "precip_dry_mm"}.issubset(df.columns):
            dry_valid = df.dropna(subset=["dry_vh_km2"])
            
            if len(dry_valid) > 0:
                region_stats["dry_season"] = {
                    "water_mean_km2": float(dry_valid["dry_vh_km2"].mean()),
                    "water_std_km2": float(dry_valid["dry_vh_km2"].std()),
                    "water_min_km2": float(dry_valid["dry_vh_km2"].min()),
                    "water_max_km2": float(dry_valid["dry_vh_km2"].max()),
                    
                    "years_analyzed": int(len(dry_valid)),
                    "data_completeness_pct": float(len(dry_valid) / len(CFG["YEARS"]) * 100),
                }
                
                # Baseline comparison
                baseline_key = region_name
                if baseline_key in BASELINES:
                    baseline_dry = BASELINES[baseline_key]["dry_km2"]
                    mean_water = dry_valid["dry_vh_km2"].mean()
                    deficit_pct = (mean_water - baseline_dry) / baseline_dry * 100
                    
                    region_stats["dry_season"]["baseline_comparison"] = {
                        "baseline_km2": float(baseline_dry),
                        "current_mean_km2": float(mean_water),
                        "deficit_pct": float(deficit_pct),
                        "status": "above_baseline" if deficit_pct > 0 else "below_baseline"
                    }
                    
                    # Tonle Sap ecological thresholds
                    if region_name == "Tonle_Sap" and "ecological_thresholds" in BASELINES[baseline_key]:
                        thresholds = BASELINES[baseline_key]["ecological_thresholds"]
                        below_moderate = (dry_valid["dry_vh_km2"] < thresholds["moderate"]).sum()
                        below_critical = (dry_valid["dry_vh_km2"] < thresholds["critical"]).sum()
                        
                        region_stats["dry_season"]["ecological_assessment"] = {
                            "thresholds": thresholds,
                            "years_below_moderate": int(below_moderate),
                            "years_below_critical": int(below_critical),
                            "violation_rate_pct": float(below_moderate / len(dry_valid) * 100),
                            "critical_rate_pct": float(below_critical / len(dry_valid) * 100)
                        }
                
                # Precipitation
                precip_valid = dry_valid.dropna(subset=["precip_dry_mm"])
                if len(precip_valid) > 0:
                    region_stats["dry_season"]["precipitation"] = {
                        "mean_mm": float(precip_valid["precip_dry_mm"].mean()),
                        "std_mm": float(precip_valid["precip_dry_mm"].std()),
                    }
        
        # ===== DECOUPLING ANALYSIS =====
        if {"precip_anomaly_pct", "water_anomaly_pct"}.issubset(df.columns):
            valid_anom = df.dropna(subset=["precip_anomaly_pct", "water_anomaly_pct"])
            
            if len(valid_anom) >= 3:
                from scipy import stats as sp_stats
                r, p_val = sp_stats.pearsonr(valid_anom["precip_anomaly_pct"], 
                                              valid_anom["water_anomaly_pct"])
                
                region_stats["decoupling"] = {
                    "correlation_r": float(r),
                    "p_value": float(p_val),
                    "significant": bool(p_val < 0.05),
                    "interpretation": "strong_coupling" if (r > 0.7 and p_val < 0.05) else
                                    "moderate_coupling" if (r > 0.4 and p_val < 0.05) else
                                    "weak_coupling" if p_val < 0.05 else
                                    "no_correlation"
                }
                
                # Quadrant analysis
                q1 = ((valid_anom["precip_anomaly_pct"] > 0) & (valid_anom["water_anomaly_pct"] > 0)).sum()
                q2 = ((valid_anom["precip_anomaly_pct"] < 0) & (valid_anom["water_anomaly_pct"] > 0)).sum()
                q3 = ((valid_anom["precip_anomaly_pct"] < 0) & (valid_anom["water_anomaly_pct"] < 0)).sum()
                q4 = ((valid_anom["precip_anomaly_pct"] > 0) & (valid_anom["water_anomaly_pct"] < 0)).sum()
                
                region_stats["decoupling"]["quadrants"] = {
                    "q1_wet_high": int(q1),
                    "q2_dry_high": int(q2),
                    "q3_dry_low": int(q3),
                    "q4_wet_low_ANOMALOUS": int(q4),
                }
                
                if q4 > 0:
                    anomalous_years = valid_anom[
                        (valid_anom["precip_anomaly_pct"] > 0) & 
                        (valid_anom["water_anomaly_pct"] < 0)
                    ]["year"].astype(int).tolist()
                    region_stats["decoupling"]["anomalous_years"] = anomalous_years
        
        stats["regions"][region_name] = region_stats
    
    return stats

# Compute statistics
summary_stats = compute_summary_statistics(df_delta, df_ts)

# Add project metadata
metadata = {
    "project": {
        "title": "Mekong River Dam Impact Analysis",
        "subtitle": "Sentinel-1 SAR Analysis (2015–2024)",
        "challenge": "NASA Space Apps Challenge 2024",
        "team": "Mekong Analysis Team",
        "generated_utc": datetime.utcnow().isoformat(),
        "version": "1.0.0"
    },
    
    "methodology": {
        "sensor": "Sentinel-1 C-band SAR",
        "polarizations": ["VV", "VH"],
        "platform": "Google Earth Engine",
        "analysis_years": CFG["YEARS"],
        "flood_months": list(CFG["FLOOD_MONTHS"]),
        "dry_months": list(CFG["DROUGHT_MONTHS"]),
        "thresholds": {
            "VV_dB": CFG["TH_VV_DB"],
            "VH_dB": CFG["TH_VH_DB"],
            "uncertainty_dB": CFG["TH_UNCERTAINTY_DB"]
        },
        "baseline_period": CFG["BASELINE_YEARS"],
        "key_events": {k: v.isoformat() for k, v in EVENTS.items()}
    },
    
    "statistics": summary_stats,
    
    "figures": [
        {
            "id": "fig_annual_flood",
            "title": "Annual Flood Extent (VV vs VH)",
            "type": "line_chart",
            "file_json": "fig_annual_flood.json",
            "file_html": "fig_annual_flood.html"
        },
        {
            "id": "fig_vh_gain_stacked",
            "title": "VH Advantage (Stacked Bar)",
            "type": "stacked_bar",
            "file_json": "fig_vh_gain_stacked.json",
            "file_html": "fig_vh_gain_stacked.html"
        },
        {
            "id": "fig_dry_season_dual",
            "title": "Dry Season Analysis (Dual-Axis)",
            "type": "dual_axis",
            "file_json": "fig_dry_season_dual.json",
            "file_html": "fig_dry_season_dual.html"
        },
        {
            "id": "fig_decoupling_scatter",
            "title": "Precipitation-Water Decoupling",
            "type": "scatter",
            "file_json": "fig_decoupling_scatter.json",
            "file_html": "fig_decoupling_scatter.html"
        }
    ],
    
    "data_quality": {
        "Mekong_Delta": {
            "flood_completeness": val_delta_flood["completeness_pct"],
            "dry_completeness": val_delta_dry["completeness_pct"],
            "issues_count": len(val_delta_flood["issues"]) + len(val_delta_dry["issues"])
        },
        "Tonle_Sap": {
            "flood_completeness": val_ts_flood["completeness_pct"],
            "dry_completeness": val_ts_dry["completeness_pct"],
            "issues_count": len(val_ts_flood["issues"]) + len(val_ts_dry["issues"])
        }
    },
    
    "key_findings": [
        {
            "title": "Dual-Polarization Advantage",
            "description": f"VH detects ~{summary_stats['regions']['Mekong_Delta']['flood_season'].get('vh_gain_mean_pct', 20):.0f}% more flooding than VV on average"
        },
        {
            "title": "Ecological Risk",
            "description": "Tonlé Sap below moderate threshold in XX% of years (fill from actual data)"
        },
        {
            "title": "Decoupling Evidence",
            "description": "Weak precipitation-water correlation proves dam control"
        }
    ]
}

# Save metadata
metadata_path = ASSETS / "dashboard_metadata.json"
with open(metadata_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"\n✅ Metadata saved → {metadata_path.name}")
print(f"   File size: {metadata_path.stat().st_size / 1024:.1f} KB")

# Display key statistics
print("\n📊 Key Statistics Summary:")
print("-" * 80)

for region_name, region_data in summary_stats["regions"].items():
    print(f"\n{region_name.replace('_', ' ')}:")
    
    if "flood_season" in region_data and region_data["flood_season"]:
        flood = region_data["flood_season"]
        print(f"   Flood Season:")
        print(f"      VH mean: {flood.get('vh_mean_km2', 0):>10,.0f} km²")
        print(f"      VH gain: {flood.get('vh_gain_mean_pct', 0):>10.1f}%")
        print(f"      Completeness: {flood.get('data_completeness_pct', 0):>7.1f}%")
    
    if "dry_season" in region_data and region_data["dry_season"]:
        dry = region_data["dry_season"]
        print(f"   Dry Season:")
        print(f"      Water mean: {dry.get('water_mean_km2', 0):>10,.0f} km²")
        
        if "baseline_comparison" in dry:
            baseline_cmp = dry["baseline_comparison"]
            print(f"      Deficit: {baseline_cmp.get('deficit_pct', 0):>10.1f}%")
        
        if "ecological_assessment" in dry:
            eco = dry["ecological_assessment"]
            print(f"      Threshold violations: {eco.get('violation_rate_pct', 0):>7.1f}%")
    
    if "decoupling" in region_data:
        dec = region_data["decoupling"]
        print(f"   Decoupling:")
        print(f"      Correlation: {dec.get('correlation_r', 0):>10.3f}")
        print(f"      Status: {dec.get('interpretation', 'unknown')}")

print("\n" + "="*80)

In [ ]:
# === Cell 13: 종합 요약 & 출력 검증 ===
"""
🎯 OBJECTIVE: Final summary and output verification

OUTPUTS:
1. All figure files (JSON + HTML)
2. Dashboard HTML
3. Metadata JSON
4. Summary text report
"""

print("\n" + "="*80)
print("NOTEBOOK 07 COMPREHENSIVE SUMMARY")
print("="*80)

# ===== OUTPUT FILES VERIFICATION =====
print("\n📁 OUTPUT FILES VERIFICATION")
print("-" * 80)

expected_outputs = {
    "Figures (JSON)": [
        "fig_annual_flood.json",
        "fig_vh_gain_stacked.json",
        "fig_dry_season_dual.json",
        "fig_decoupling_scatter.json"
    ],
    "Figures (HTML)": [
        "fig_annual_flood.html",
        "fig_vh_gain_stacked.html",
        "fig_dry_season_dual.html",
        "fig_decoupling_scatter.html"
    ],
    "Dashboard": [
        "dashboard.html"
    ],
    "Metadata": [
        "dashboard_metadata.json"
    ],
    "Cached Data": [
        "annual_flood_mekong_delta.csv",
        "annual_flood_tonle_sap.csv",
        "annual_dry_mekong_delta.csv",
        "annual_dry_tonle_sap.csv"
    ]
}

all_exist = True
total_size_kb = 0

for category, files in expected_outputs.items():
    print(f"\n{category}:")
    category_exists = True
    
    for fname in files:
        fpath = ASSETS / fname
        
        if fpath.exists():
            size_kb = fpath.stat().st_size / 1024
            total_size_kb += size_kb
            print(f"   ✓ {fname:<45} ({size_kb:>7.1f} KB)")
        else:
            print(f"   ❌ {fname:<45} (MISSING)")
            category_exists = False
            all_exist = False
    
    if not category_exists:
        print(f"   ⚠️  Some {category} files missing")

print(f"\n{'─' * 80}")
print(f"Total output size: {total_size_kb:,.1f} KB ({total_size_kb/1024:.2f} MB)")

if all_exist:
    print("\n✅ All expected outputs generated successfully")
else:
    print("\n⚠️  Some outputs missing — review cell execution")

# ===== SUMMARY STATISTICS =====
print("\n" + "="*80)
print("📊 ANALYSIS SUMMARY")
print("="*80)

print(f"\nAnalysis Period: {min(CFG['YEARS'])}-{max(CFG['YEARS'])} ({len(CFG['YEARS'])} years)")
print(f"Flood Season: {CFG['FLOOD_MONTHS'][0]:02d}-{CFG['FLOOD_MONTHS'][1]:02d} (Aug-Sep)")
print(f"Dry Season: {CFG['DROUGHT_MONTHS'][0]:02d}-{CFG['DROUGHT_MONTHS'][1]:02d} (Mar-Apr)")

print("\nRegions Analyzed:")
print("   • Mekong Delta (Vietnam)")
print("   • Tonlé Sap (Cambodia)")

print("\nKey Datasets:")
for name, df in [("Delta Flood", df_delta_flood), ("Delta Dry", df_delta_dry),
                  ("Tonlé Flood", df_ts_flood), ("Tonlé Dry", df_ts_dry)]:
    completeness = (1 - df.isna().sum().sum() / (len(df) * len(df.columns))) * 100 if len(df) > 0 else 0
    print(f"   • {name:12}: {len(df):>2} rows, {completeness:>5.1f}% complete")

# ===== KEY FINDINGS =====
print("\n" + "="*80)
print("🎯 KEY FINDINGS")
print("="*80)

findings = []

# Finding 1: VH Advantage
if "flood_season" in summary_stats["regions"]["Mekong_Delta"]:
    delta_gain = summary_stats["regions"]["Mekong_Delta"]["flood_season"].get("vh_gain_mean_pct", 0)
    findings.append(f"VH Advantage: ~{delta_gain:.0f}% more flood detection than VV")

# Finding 2: Ecological Risk
if "dry_season" in summary_stats["regions"]["Tonle_Sap"]:
    if "ecological_assessment" in summary_stats["regions"]["Tonle_Sap"]["dry_season"]:
        eco = summary_stats["regions"]["Tonle_Sap"]["dry_season"]["ecological_assessment"]
        violation_rate = eco.get("violation_rate_pct", 0)
        findings.append(f"Ecological Risk: {violation_rate:.0f}% of years below Tonlé Sap threshold")

# Finding 3: Decoupling
decoupling_found = False
for region in ["Mekong_Delta", "Tonle_Sap"]:
    if "decoupling" in summary_stats["regions"][region]:
        r = summary_stats["regions"][region]["decoupling"].get("correlation_r", 0)
        if abs(r) < 0.5:
            findings.append(f"Decoupling: {region.replace('_', ' ')} r={r:.2f} (weak coupling = dam control)")
            decoupling_found = True
            break

if findings:
    for i, finding in enumerate(findings, 1):
        print(f"\n   {i}. {finding}")
else:
    print("\n   ⚠️  Insufficient data for key findings")

# ===== RECOMMENDATIONS =====
print("\n" + "="*80)
print("💡 RECOMMENDATIONS FOR NASA PRESENTATION")
print("="*80)

recommendations = [
    ("Lead with Impact", 
     "Start with Tonlé Sap ecological crisis (visual + emotional hook)"),
    
    ("Show VH Advantage", 
     "Use stacked bar chart to demonstrate 'hidden' flooding under vegetation"),
    
    ("Prove Dam Control", 
     "Decoupling scatter plot = smoking gun (physically impossible without dams)"),
    
    ("Quantify Uncertainty", 
     "Always show ranges/error bars (builds scientific credibility)"),
    
    ("Interactive Demo", 
     "Use dashboard.html for live exploration during Q&A"),
    
    ("Call to Action", 
     "End with policy recommendations (transboundary monitoring, ecological flows)")
]

for i, (title, description) in enumerate(recommendations, 1):
    print(f"\n   {i}. {title}")
    print(f"      → {description}")

# ===== NEXT STEPS =====
print("\n" + "="*80)
print("🚀 NEXT STEPS")
print("="*80)

next_steps = [
    "Deploy dashboard to Streamlit Cloud / GitHub Pages",
    "Create 30-second video using dashboard visualizations",
    "Prepare 7-slide deck (use exported figures)",
    "Write README.md with installation instructions",
    "Tag GitHub release v1.0.0",
    "Submit to NASA Space Apps portal"
]

for i, step in enumerate(next_steps, 1):
    print(f"   {i}. {step}")

# ===== FINAL CHECKLIST =====
print("\n" + "="*80)
print("✅ FINAL CHECKLIST")
print("="*80)

checklist = {
    "Data processing complete": all_exist,
    "All figures generated": all((ASSETS / f"{fig}.json").exists() for fig in 
                                  ["fig_annual_flood", "fig_vh_gain_stacked", "fig_dry_season_dual", "fig_decoupling_scatter"]),
    "Dashboard HTML created": (ASSETS / "dashboard.html").exists(),
    "Metadata JSON saved": (ASSETS / "dashboard_metadata.json").exists(),
    "Data quality validated": avg_completeness >= 70 if 'avg_completeness' in locals() else False,
    "Statistics computed": len(summary_stats["regions"]) == 2,
}

for item, status in checklist.items():
    icon = "✅" if status else "❌"
    print(f"   {icon} {item}")

if all(checklist.values()):
    print("\n🎉 ALL SYSTEMS GO — Ready for presentation!")
else:
    print("\n⚠️  Some tasks incomplete — review above")

# ===== GENERATE SUMMARY TXT =====
summary_text = f"""
[Notebook 07 Summary — Interactive Visualizations & Dashboard]

Generated: {datetime.now().isoformat()}
Analysis Period: {min(CFG['YEARS'])}-{max(CFG['YEARS'])}

OUTPUTS:
- 4 interactive figures (JSON + HTML)
- Standalone dashboard (dashboard.html)
- Metadata JSON (dashboard_metadata.json)
- Cached datasets (4 CSV files)

TOTAL SIZE: {total_size_kb:,.1f} KB ({total_size_kb/1024:.2f} MB)

KEY FEATURES:
1. Responsive design (mobile-friendly)
2. Offline-ready (all assets embedded)
3. Accessibility (ARIA labels, keyboard navigation)
4. Scientific rigor (error bars, p-values, uncertainty ranges)

DASHBOARD SECTIONS:
- Project Overview
- Annual Flood Extent (VV vs VH)
- VH Polarization Advantage
- Dry Season Impact
- Precipitation-Water Decoupling
- Key Findings & Recommendations
- Technical Details

NEXT ACTIONS:
1. Open dashboard.html in browser
2. Review all interactive figures
3. Test on mobile device
4. Deploy to web hosting (Streamlit/GitHub Pages)
5. Create presentation materials

NASA SPACE APPS DELIVERABLES:
✓ Interactive dashboard
✓ Scientific visualizations
✓ Reproducible code
✓ Open data
✓ Documentation

CONTACT:
For questions or collaboration: your-email@example.com
GitHub: https://github.com/your-repo/mekong-analysis
"""

summary_path = ASSETS / "notebook07_summary.txt"
summary_path.write_text(summary_text, encoding="utf-8")

print(f"\n💾 Summary saved → {summary_path.name}")

print("\n" + "="*80)
print("✅ NOTEBOOK 07 COMPLETE — INTERACTIVE VISUALIZATIONS READY")
print("="*80)

print(f"\n📂 All outputs in: {ASSETS.relative_to(NB_DIR)}")
print(f"🌐 Open dashboard: file://{(ASSETS / 'dashboard.html').absolute()}")
print(f"📊 Figures available in both JSON (for embedding) and HTML (standalone)")

print("\n🎯 Remember for presentation:")
print("   • Lead with ecological impact (Tonlé Sap crisis)")
print("   • Show VH advantage visually (stacked bars)")
print("   • Prove dam control (decoupling scatter)")
print("   • Always mention uncertainties (builds credibility)")
print("   • End with policy recommendations")

print("\n🚀 Ready to change the Mekong! 🌊")
print("="*80)

In [ ]:
# === Cell 14 (BONUS): Timeline Visualization ===
"""
🎯 OBJECTIVE: Create event timeline for presentation

OPTIONAL: Chronological view of key events and impacts
"""

print("\n" + "="*80)
print("BONUS: CREATING EVENT TIMELINE")
print("="*80)

def create_timeline_figure():
    """
    Create interactive timeline of key events and impacts.
    
    Returns:
        plotly Figure
    """
    events_data = [
        {"year": 2009, "event": "Xiaowan Dam Online", "type": "dam", "impact": "moderate"},
        {"year": 2012, "event": "Nuozhadu Dam Online", "type": "dam", "impact": "moderate"},
        {"year": 2019, "event": "Jinghong Flow Cut", "type": "crisis", "impact": "severe"},
    ]
    
    # Add data-driven events
    if len(df_ts) > 0 and "dry_vh_km2" in df_ts.columns:
        ts_dry_valid = df_ts.dropna(subset=["dry_vh_km2"])
        
        if "ecological_thresholds" in BASELINES.get("Tonle_Sap", {}):
            threshold = BASELINES["Tonle_Sap"]["ecological_thresholds"]["moderate"]
            
            for _, row in ts_dry_valid.iterrows():
                if row["dry_vh_km2"] < threshold:
                    events_data.append({
                        "year": int(row["year"]),
                        "event": "Below Ecological Threshold",
                        "type": "impact",
                        "impact": "critical" if row["dry_vh_km2"] < 2000 else "severe"
                    })
    
    # Create figure
    fig = go.Figure()
    
    # Timeline bar
    fig.add_trace(go.Scatter(
        x=[2005, 2024],
        y=[0, 0],
        mode="lines",
        line=dict(color="gray", width=3),
        showlegend=False,
        hoverinfo="skip"
    ))
    
    # Events
    colors = {
        "dam": COLORS["primary"],
        "crisis": COLORS["danger"],
        "impact": COLORS["moderate"]
    }
    
    for event in events_data:
        color = colors.get(event["type"], "gray")
        marker_size = 20 if event["impact"] == "severe" else 15 if event["impact"] == "moderate" else 10
        
        fig.add_trace(go.Scatter(
            x=[event["year"]],
            y=[0],
            mode="markers+text",
            marker=dict(size=marker_size, color=color, line=dict(width=2, color="white")),
            text=event["event"],
            textposition="top center",
            textfont=dict(size=9),
            name=event["type"].capitalize(),
            hovertemplate=f"<b>{event['year']}</b><br>{event['event']}<br>Impact: {event['impact']}<extra></extra>"
        ))
    
    fig.update_layout(
        title="Timeline: Key Events & Impacts (2005–2024)",
        xaxis=dict(title="Year", range=[2004, 2025], dtick=2),
        yaxis=dict(visible=False),
        height=300,
        showlegend=True,
        hovermode="closest",
        font=dict(family="Arial, sans-serif", size=12),
        plot_bgcolor="white",
        paper_bgcolor="white"
    )
    
    fig.update_xaxes(showgrid=True, gridcolor="lightgray")
    
    # Save
    fig.write_json(str(ASSETS / "fig_timeline.json"))
    fig.write_html(str(ASSETS / "fig_timeline.html"))
    
    print("   ✓ Timeline figure created")
    print("   💾 Saved → fig_timeline.json")
    print("   💾 Saved → fig_timeline.html")
    
    return fig

try:
    fig_timeline = create_timeline_figure()
    fig_timeline.show()
    print("\n✅ Timeline visualization complete")
except Exception as e:
    print(f"\n⚠️  Timeline creation failed: {type(e).__name__}: {e}")

print("\n" + "="*80)